In [ ]:
pdfFile = '/Catalogues/Catalogue Spanish_3.pdf'

In [ ]:
# import gc
# import os
# import fitz
# # Read the PDF file
# doc = fitz.open(pdfFile)
# print('page','Δ', 'block number','Δ', 'size','Δ', 'flags', 'Δ', 'font','Δ', 'color','Δ', 'ascender', 'descender','Δ', 'text','Δ', 'line_bbox','Δ', 'line','Δ', 'block')

# # Iterate through the pages
# for i in range(len(doc)):
    
#     page = doc[i]

#     for block in page.get_text('dict')['blocks']:
#         # check if all lines in block have the same color, font and aproximatelly the same size
#         if block['type'] == 0:
#             print(block['number'])
#         if 'lines' in block:
#             for line in block['lines']:
#                 for span in line['spans']:
#                     print(i,'Δ', block['number'],'Δ', span['size'],'Δ', span['flags'], 'Δ', span['font'],'Δ', span['color'],'Δ', span['ascender'], span['descender'],'Δ', span['text'],'Δ', line['bbox'],'Δ', line,'Δ', block)

In [ ]:
import fitz
import pprint

def find_keys_in_blocks(pgList='all', pdfFile=pdfFile, analysis_type='both'):
    """ This function takes:
    - a list of pages (pgList) or 'all' to iterate through all the pages
    - a pdf file (pdfFile)
    - the type of analysis (analysis_type): 'text', 'image' or 'both'
    and returns a dictionary with the keys in the blocks of the pages and the number of times they appear in the page 
    and the pages they appear"""
    keys = {'image':{'blocks':{}}, 'text':{'blocks':{}, 'lines':{}, 'spans':{}}}

    doc = fitz.open(pdfFile)

    if pgList == 'all':
        pgList = range(len(doc))

    for i in pgList:
        page = doc[i]
        for block in page.get_text('dict')['blocks']:
            if block['type'] == 0 and analysis_type in ['text', 'both']:  # Text block
                for key in block.keys():
                    if key not in keys['text']['blocks'].keys():
                        keys['text']['blocks'][key] = [1, [i]]
                    else:
                        keys['text']['blocks'][key][0] += 1
                        if i not in keys['text']['blocks'][key][1]:
                            keys['text']['blocks'][key][1].append(i)
                for line in block['lines']:
                    for key in line.keys():
                        if key not in keys['text']['lines'].keys():
                            keys['text']['lines'][key] = [1, [i]]
                        else:
                            keys['text']['lines'][key][0] += 1
                            if i not in keys['text']['lines'][key][1]:
                                keys['text']['lines'][key][1].append(i)
                    for span in line['spans']:
                        for key in span.keys():
                            if key not in keys['text']['spans'].keys():
                                keys['text']['spans'][key] = [1, [i]]
                            else:
                                keys['text']['spans'][key][0] += 1
                                if i not in keys['text']['spans'][key][1]:
                                    keys['text']['spans'][key][1].append(i)
            elif block['type'] == 1 and analysis_type in ['image', 'both']:  # Image block
                for key in block.keys():
                    if key not in keys['image']['blocks'].keys():
                        keys['image']['blocks'][key] = [1, [i]]
                    else:
                        keys['image']['blocks'][key][0] += 1
                        if i not in keys['image']['blocks'][key][1]:
                            keys['image']['blocks'][key][1].append(i)
    # close the PDF file
    doc.close()                        
    return keys

keys_summary = find_keys_in_blocks(pgList='all', pdfFile=pdfFile, analysis_type='both')
# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(keys_summary)

# print all keys in keys_summary dictionary and subdictionaries
for key in keys_summary.keys():
    print(key)
    if type(keys_summary[key]) == dict:
        for subkey in keys_summary[key].keys():
            print('\t', subkey)
            if type(keys_summary[key][subkey]) == dict:
                for subsubkey in keys_summary[key][subkey].keys():
                    print('\t\t', subsubkey, ':', keys_summary[key][subkey][subsubkey][0])



# Get the different combinations of values in '''keys_summary[key][subkey]'' throughout the document, also retrieving the text itself


In [ ]:
def find_text_patterns(pgList='all', pdfFile=pdfFile, span_attributes=['size', 'flags', 'font', 'color']):
    patterns = {}

    doc = fitz.open(pdfFile)

    if pgList == 'all':
        pgList = range(len(doc))

    for i in pgList:
        page = doc[i]
        for block in page.get_text('dict')['blocks']:
            if block['type'] == 0:  # Text block
                for line in block['lines']:
                    for span in line['spans']:
                        if span_attributes == 'all':
                            attributes = tuple(span.keys())
                        else:
                            attributes = tuple([span[key] for key in span_attributes])
                        if attributes not in patterns:
                            patterns[attributes] = [(span['text'], block['bbox'], [i])]
                        else:
                            # Check if the text and position are the same as the last entry for these attributes
                            if patterns[attributes][-1][:2] == (span['text'], block['bbox']):
                                # If they are, append the page number to the list in the last entry
                                patterns[attributes][-1][2].append(i)
                            else:
                                # If they're not, add a new entry with the text, position, and a list with the page number
                                patterns[attributes].append((span['text'], block['bbox'], [i]))
    # close the PDF file
    doc.close()
    return patterns

text_patterns = find_text_patterns(pgList='all', pdfFile=pdfFile,span_attributes=['size', 'flags', 'font', 'color'])

print(len(text_patterns))
# Print the patterns
for attributes, texts_positions in text_patterns.items():
    print(f'Attributes: {attributes} --> Number of occurrences: {len(texts_positions)}')
    for text, position, pages in texts_positions:
        print(f'\tText: {text}, Position: {position}, Pages: {pages} --> {len(pages)}')

In [ ]:
unique_dict = {}
for attributes, texts_positions in text_patterns.items():
    # print(f'Attributes: {attributes[2]}')
    unique_dict[attributes[2]] = []
for attr in unique_dict.keys():
    print(attr)

In [ ]:
# create a dict of the different positions of the atribute combinations, and the pages they appear
attr_pos_dict = {}
for attributes, texts_positions in text_patterns.items():
    # print(texts_positions)
    # print(f'Attributes: {attributes} --> Number of occurrences: {len(texts_positions)}')
    if attributes not in attr_pos_dict.keys():
        attr_pos_dict[attributes] = {position: [page] for text, position, page in texts_positions}
    for text, position, page in texts_positions:
        if position not in attr_pos_dict[attributes]:
            attr_pos_dict[attributes][position] = [page]
        else:
            attr_pos_dict[attributes][position].append(page)

# print the positions
for key_attributes, val_atributes in attr_pos_dict.items():
    print(f'\nAttributes: {key_attributes} --> Number of occurrences: {len(val_atributes)}')
    for key_position, val_page in val_atributes.items():
        print(f'\tPosition: {key_position} --> Number of occurrences: {len(val_page)}')
        # create a list of the pages where the position appears
        pg_list  = [x for x in val_page]
        # flatten pg_list
        pg_list  = [val for sublist in pg_list for val in sublist]        
        print(f'\t\tPages: {pg_list}')
        # for page in val_page:
        #     print(f'\t\tPage: {page}')


In [ ]:
import fitz
from PIL import Image
import IPython.display as display

#------------------------------------------------------------------------------------
# 1 -\\\\\\\\\\\- Remove Anotations (text, and images) -\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#------------------------------------------------------------------------------------
def remove_annotations(page, extractictionType='text'):
    ''' Requires a page object from fitz.open() 
                            and the type of extraction ('text', 'image' or 'both')'''
    # Get the list of blocks
    blocks = page.get_text('dict')['blocks']

    # Count the number of image blocks
    num_image_blocks = sum(1 for block in blocks if 'image' in block)

    # Count the number of text blocks
    num_text_blocks = sum(1 for block in blocks if 'lines' in block)

    if extractictionType == 'text':
        if num_text_blocks == 0:
            return page
        elif num_image_blocks == 0:
            threshold = 2
        elif num_image_blocks > 0:
            threshold = num_text_blocks - num_image_blocks
    elif extractictionType == 'image' :
        if num_image_blocks == 0:
            return page
        elif num_text_blocks == 0:
            threshold = 0
            page_bbox = fitz.Rect(page.rect)
            page.add_redact_annot(page_bbox)
            page.apply_redactions()
        elif num_text_blocks > 0:
            # if num_image_blocks > num_text_blocks:
            #     threshold = num_image_blocks - num_text_blocks
            # threshold = abs(num_image_blocks - num_text_blocks)
            """ for this to work i would need to create an incremental threshold, uppon each iteration the threshold would increase by 1, and the while loop would stop when the threshold is reached """
    elif extractictionType == 'both':
        threshold = 0
        page_bbox = fitz.Rect(page.rect)
        page.add_redact_annot(page_bbox)
        page.apply_redactions()
    else:
        return page

    # Remove all other annotations from the page
    while len(page.get_text('dict')['blocks'])>threshold:
        for block in page.get_text('dict')['blocks']:
            if 'lines' in block and extractictionType == 'text' or 'lines' in block and extractictionType == 'both':
                for line in block['lines']:
                    for span in line['spans']:
                        searchText = span['text']
                        # Search for the text
                        text_instances = page.search_for(searchText)
                        # Redact the text
                        for inst in text_instances:
                            page.add_redact_annot(inst)
                            page.apply_redactions()
            elif 'image' in block and extractictionType == 'image' or 'image' in block and extractictionType == 'both':
                # If the block is an image, redact it
                image_bbox = fitz.Rect(block['bbox'])
                page.add_redact_annot(image_bbox)
                page.apply_redactions()
        # print(page.get_text('dict')['blocks'])
        # page.draw_rect(block['bbox'], color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)
    # # Save the redacted PDF
    # doc.save('redacted_fullText.pdf', garbage=4, deflate=True, clean=True)
    return page

#------------------------------------------------------------------------------------
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
# ------------------------------------------------------------------------------------

## get the <b>bboxes of the images</b> in the 'infographic glossary'

In [ ]:
import cv2
import fitz
import numpy as np
import matplotlib.pyplot as plt



# Function to merge close rectangles
def merge_close_rectangles(bboxes, threshold):
    # Function to calculate distance between two rectangles
    def distance(r1, r2):
        # find if bboxes are intersecting
        if r1[0] < r2[0] + r2[2] and r1[0] + r1[2] > r2[0] and r1[1] < r2[1] + r2[3] and r1[1] + r1[3] > r2[1]:
            return 0, 0
        # get the x or y distance between the two rectangles if they are not intersecting
        x1, y1, w1, h1 = r1
        x2, y2, w2, h2 = r2
        return min(abs(x1 - x2), abs(x1 + w1 - x2 - w2)), min(abs(y1 + h1 - y2 - h2), abs(y1 - y2))#minx, miny

    # Function to merge a list of rectangles into one
    def merge(rects):
        x_min = min(r[0] for r in rects)
        y_min = min(r[1] for r in rects)
        x_max = max(r[0] + r[2] for r in rects)
        y_max = max(r[1] + r[3] for r in rects)
        return (x_min, y_min, x_max - x_min, y_max - y_min)

    merged = True
    while merged:
        merged = False
        for i in range(len(bboxes)):
            for j in range(i + 1, len(bboxes)):
                if distance(bboxes[i], bboxes[j])[0] < threshold and distance(bboxes[i], bboxes[j])[1] < threshold:
                    bboxes[i] = merge([bboxes[i], bboxes[j]])
                    del bboxes[j]
                    merged = True
                    break
            if merged:
                break

    return bboxes


# Function to check if a rectangle is inside another
def is_inside(r1, r2):
    x1, y1, w1, h1 = r1
    x2, y2, w2, h2 = r2
    return x1 > x2 and y1 > y2 and x1 + w1 < x2 + w2 and y1 + h1 < y2 + h2

# Function to calculate area of a rectangle
def area(bbox):
    _, _, w, h = bbox
    return w * h

def cv2_to_fitz(bbox):
    x, y, w, h = bbox
    return fitz.Rect(x, y, x + w, y + h)


# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[7]

# Remove all annotations from the page
page = remove_annotations(page, extractictionType='both')

# Get the dimensions of the page
dimensions = page.rect
clip_areas = [dimensions]#[fitz.Rect(605, 515, 805, 565),fitz.Rect(815, 515, 1030, 565)]

for clip_area in clip_areas:
    # Get the pixmap
    pix = page.get_pixmap(clip=clip_area)

    # Convert the pixmap to a numpy array
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)

    # Convert RGB to BGR
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


    # Convert the image to grayscale
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Apply the thresholding  
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)#(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # Get the bounding boxes
    bboxes = [cv2.boundingRect(contour) for contour in contours]

    # Remove rectangles that contain all others
    bboxes = [r for r in bboxes if any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Remove rectangles that are contained in others
    bboxes = [r for r in bboxes if not any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Use the function on your bounding boxes
    bboxes = merge_close_rectangles(bboxes, 10)  # Adjust the threshold as needed

    # Filter out rectangles with area < 90
    bboxes = [bbox for bbox in bboxes if area(bbox) >= 399]

    # Filter out rectangles with area > 850
    bboxes = [bbox for bbox in bboxes if area(bbox) <= 901]

    # Convert the bounding boxes to fitz.Rect
    image_fitz_bboxes = [cv2_to_fitz(bbox) for bbox in bboxes]

    # Draw the bounding boxes on the image for visualization
    for bbox in bboxes:
        x, y, w, h = bbox
        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
        area = w*h
        # cv2.putText(img_bgr, str(area), (x+50, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)

# close the document
doc.close()

import cv2
import fitz
import numpy as np
import matplotlib.pyplot as plt



# Function to merge close rectangles
def merge_close_rectangles(bboxes, threshold):
    # Function to calculate distance between two rectangles
    def distance(r1, r2):
        # find if bboxes are intersecting
        if r1[0] < r2[0] + r2[2] and r1[0] + r1[2] > r2[0] and r1[1] < r2[1] + r2[3] and r1[1] + r1[3] > r2[1]:
            return 0, 0
        # get the x or y distance between the two rectangles if they are not intersecting
        x1, y1, w1, h1 = r1
        x2, y2, w2, h2 = r2
        return min(abs(x1 - x2), abs(x1 + w1 - x2 - w2)), min(abs(y1 + h1 - y2 - h2), abs(y1 - y2))#minx, miny

    # Function to merge a list of rectangles into one
    def merge(rects):
        x_min = min(r[0] for r in rects)
        y_min = min(r[1] for r in rects)
        x_max = max(r[0] + r[2] for r in rects)
        y_max = max(r[1] + r[3] for r in rects)
        return (x_min, y_min, x_max - x_min, y_max - y_min)

    merged = True
    while merged:
        merged = False
        for i in range(len(bboxes)):
            for j in range(i + 1, len(bboxes)):
                if distance(bboxes[i], bboxes[j])[0] < threshold and distance(bboxes[i], bboxes[j])[1] < threshold:
                    bboxes[i] = merge([bboxes[i], bboxes[j]])
                    del bboxes[j]
                    merged = True
                    break
            if merged:
                break

    return bboxes


# Function to check if a rectangle is inside another
def is_inside(r1, r2):
    x1, y1, w1, h1 = r1
    x2, y2, w2, h2 = r2
    return x1 > x2 and y1 > y2 and x1 + w1 < x2 + w2 and y1 + h1 < y2 + h2

# Function to calculate area of a rectangle
def area(bbox):
    _, _, w, h = bbox
    return w * h

def cv2_to_fitz(bbox):
    x, y, w, h = bbox
    return fitz.Rect(x, y, x + w, y + h)


# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[7]

# Remove all annotations from the page
page = remove_annotations(page, extractictionType='both')

# Get the dimensions of the page
dimensions = page.rect
clip_areas = [dimensions]#[fitz.Rect(605, 515, 805, 565),fitz.Rect(815, 515, 1030, 565)]

for clip_area in clip_areas:
    # Get the pixmap
    pix = page.get_pixmap(clip=clip_area)

    # Convert the pixmap to a numpy array
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)

    # Convert RGB to BGR
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


    # Convert the image to grayscale
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Apply the thresholding  
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)#(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # Get the bounding boxes
    bboxes = [cv2.boundingRect(contour) for contour in contours]

    # Remove rectangles that contain all others
    bboxes = [r for r in bboxes if any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Remove rectangles that are contained in others
    bboxes = [r for r in bboxes if not any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Use the function on your bounding boxes
    bboxes = merge_close_rectangles(bboxes, 10)  # Adjust the threshold as needed

    # Filter out rectangles with area < 90
    bboxes = [bbox for bbox in bboxes if area(bbox) >= 399]

    # Filter out rectangles with area > 850
    bboxes = [bbox for bbox in bboxes if area(bbox) <= 901]

    #  Filter bboxes that are within x0 170, x1 760, y0 170, y1 530
    bboxes = [bbox for bbox in bboxes if bbox[0] > 170 and bbox[0] + bbox[2] < 760 and bbox[1] > 170 and bbox[1] + bbox[3] < 530]

    # Convert the bounding boxes to fitz.Rect
    image_fitz_bboxes = [cv2_to_fitz(bbox) for bbox in bboxes]

    # Draw the bounding boxes on the image for visualization
    for bbox in bboxes:
        x, y, w, h = bbox
        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
        area = w*h
        # cv2.putText(img_bgr, str(area), (x+50, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        cv2.putText(img_bgr, str((x, y, w, h)), (x+50, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (36,255,12), 2)






    # Use plt.imshow to display the image
    plt.imshow(img_bgr)
    plt.show()

    """ # Display the image with bounding boxes
    cv2.imshow('Image with Bounding Boxes', img_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows() """

# close the document
doc.close()

# clear all memory but 'image_fitz_bboxes'
del globals()['doc']
del globals()['page']
del globals()['dimensions']
del globals()['clip_areas']
del globals()['pix']
del globals()['img']
del globals()['img_bgr']
del globals()['gray']
del globals()['thresh']
del globals()['contours']
del globals()['bboxes']
del globals()['bbox']
del globals()['x']
del globals()['y']
del globals()['w']
del globals()['h']
del globals()['clip_area']

# Run the garbage collector
import gc
gc.collect()







## get the <b>bboxes of the image labels</b>

In [ ]:
import fitz
from PIL import Image,ImageDraw
import io
from IPython import display

# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[7]

# 1st -------------------------------------------------------
# Remove necessary text to get the bboxes of the image labels
# ------------------------------------------------------------
# Get the list of blocks
""" For the display, since pymupdf draw_rect is not erasable we use a pil image of the page """
# render page to an image
pix0 = page.get_pixmap()

# Get the image data
img_data = pix0.tobytes("png")

# Create a PIL image from the data
img0 = Image.open(io.BytesIO(img_data))

# create a drawing context for PIL
draw = ImageDraw.Draw(img0)

for block in page.get_text('dict')['blocks']:
    # print(block)
    if 'lines' in block:
        link_text = ''
        for line in block['lines']:
            for span in line['spans']:
                # {'size': 9.0, 'flags': 4, 'font': 'Montserrat-Regular', 'color': 0
                if span['size'] != 9.0 or span['flags'] != 4 or span['font'] != 'Montserrat-Regular' or span['color'] != 0:
                    print('here')
                    searchText = span['text']
                    print('searchText', searchText)
                    # Search for the text
                    text_instances = page.search_for(searchText)

                    print('text_instances', text_instances)

                    # Redact the text - there is a internal link with text 'Table of Contents |' that is not being redacted
                    text_instances += page.search_for(' Table of Contents | ')
                    for inst in text_instances:
                        page.add_redact_annot(inst)
                    page.apply_redactions()
                    
                    # Draw a rectangle on the image
                    # The arguments are two points of the diagonal of the rectangle
                    draw.rectangle(block['bbox'], outline="red")
                    # page.draw_rect(block['bbox'], color=(1, 0, 0), width=0.3)


                else:
                    draw.rectangle(block['bbox'], outline="green")
                    # page.draw_rect(block['bbox'], color=(0, 1, 0), width=0.7)#, fill=(1, 0, 0), width=0.3, overlay=True)

# Display the image - PIL representation of the page
display.display(img0)

# Display the image - The resluts of the redaction in the page object
# Render the page to an image
pix = page.get_pixmap()
# Get the image data from the pixmap
data = pix.samples
# Create a PIL image from the data
img = Image.frombytes("RGB", [pix.width, pix.height], data)
# Display the image
display.display(img)
""" # Save the redacted PDF
doc.save('redacted.pdf', garbage=4, deflate=True, clean=True) """




# 2nd -------------------------------------------------------
# Get the bboxes of the image labels
# ------------------------------------------------------------

textBboxes = []#image_fitz_bboxes
for block in page.get_text('dict')['blocks']:
    if 'lines' in block:
        # print(block['lines'])
        textBboxes.append((block['bbox']))
    # if block['number'] >1:
    #     if block['number'] == 2:
    #         cult = (block['lines'][0]['spans'][0]['bbox'][0], block['lines'][0]['spans'][0]['bbox'][1], block['lines'][1]['spans'][0]['bbox'][2], block['lines'][1]['spans'][0]['bbox'][3])
    #         textBboxes.append(cult)
    #         access = block['lines'][2]['spans'][0]['bbox']
    #         textBboxes.append(access)
    #     else:
    #         textBboxes.append((block['bbox']))
print('---')
print(textBboxes)

textBboxes = textBboxes[0:-1]

label_dict = {}
# Draw a rectangle on the page for each bbox
for bbox in textBboxes:
    rect = fitz.Rect(*bbox)
    text = page.get_textbox(rect)
    text_single_line = text.replace('\n', '')
    label_dict[bbox] = text_single_line
    # print(text_single_line)
    page.draw_rect(bbox, color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)

# Render the page to an image
pix = page.get_pixmap()
# Get the image data from the pixmap
data = pix.samples
# Create a PIL image from the data
img = Image.frombytes("RGB", [pix.width, pix.height], data)
# Display the image
display.display(img)

# close the document
doc.close()


# clear all memory but 'textBboxes'
del globals()['doc']
del globals()['page']
del globals()['pix0']
del globals()['img_data']
del globals()['img0']
del globals()['draw']
del globals()['block']
del globals()['text_instances']
del globals()['inst']
del globals()['pix']
del globals()['data']
del globals()['img']
del globals()['rect']
del globals()['text']
del globals()['text_single_line']
del globals()['bbox']
# del globals()['cult']
# del globals()['access']
del globals()['span']
del globals()['line']
del globals()['searchText']
del globals()['link_text']


# Run the garbage collector
import gc
gc.collect()


In [ ]:
print(label_dict)

## extract and <b>label the images</b>

In [ ]:
import os
def get_closest_left_bbox(text_bboxes, img_bboxes):
    closest_left_bboxes = []
    for text_bbox in text_bboxes:
        min_distance = float('inf')
        closest_left_bbox = None
        for img_bbox in img_bboxes:
            if img_bbox[0] < text_bbox[0]:  # Check if the image bbox is to the left of the text bbox
                # Calculate the distance between the centers of the two bboxes
                distance = ((img_bbox[0] + img_bbox[2]/2 - (text_bbox[0] + text_bbox[2]/2))**2 +
                            (img_bbox[1] + img_bbox[3]/2 - (text_bbox[1] + text_bbox[3]/2))**2)**0.5
                if distance < min_distance:
                    min_distance = distance
                    closest_left_bbox = img_bbox
        closest_left_bboxes.append((text_bbox, closest_left_bbox))
    return closest_left_bboxes

""" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! """
# Check if the text and image bboxes are the same lenght
# should be the same lenght, if not please rerun the code
print(len(textBboxes)) # --> cell 32
print(len(image_fitz_bboxes)) #  --> cell 29
# Uncomment below to see the bboxes diferences
""" for i, box in enumerate(textBboxes):
    #  draw the text bboxes
    page.draw_rect(box, color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)

    #  write 'i' in the image next to the text bbox
    page.insert_text((box[2] + 5, box[1] + 10), str(i), fontsize=10, color=(1, 0, 0))

for i, box in enumerate(image_fitz_bboxes):
    #  draw the image bboxes
    page.draw_rect(box, color=(0, 1, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)

    #  write 'i' in the image next to the text bbox
    page.insert_text((box[0] - 15, box[1] + 10), str(i), fontsize=10, color=(0, 1, 0)) """

""" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! """

# Order the image_fitz_bboxes to the order of the textBboxes list
images = get_closest_left_bbox(textBboxes, image_fitz_bboxes)
# print(image_fitz_bboxes)
print(images)

# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[7]

for i, bbox in enumerate(images):
    text_bbox, closest_left_bbox = bbox
    #  draw the text bboxes
    page.draw_rect(text_bbox, color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)
    #  write 'i' in the image next to the text bbox
    page.insert_text((text_bbox[2] + 5, text_bbox[1] + 10), str(i), fontsize=10, color=(1, 0, 0))

    # draw the image bboxes
    page.draw_rect(closest_left_bbox, color=(0, 1, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)
    #  write 'i' in the image next to the text bbox
    page.insert_text((closest_left_bbox[0] - 15, closest_left_bbox[1] + 10), str(i), fontsize=10, color=(0, 1, 0))

# Render the page to an image
pix = page.get_pixmap()
# Get the image data from the pixmap
data = pix.samples
# Create a PIL image from the data
img = Image.frombytes("RGB", [pix.width, pix.height], data)
# Display the image
display.display(img)

# Close the document
doc.close()


# glossary dictionary
glossDict = {}

# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[7]

for bbox in images:
    text_bbox, closest_left_bbox = bbox

    # get the test to the left of '/' in label_dict[text_bbox]
    simplified_name = '_'.join(label_dict[text_bbox].split(':')[0].split('/')[0].split('(/)')[0].strip().split(' '))

    # check if folder f'1stSpanish/3rd_ed_trainingData/{simplified_name}' exists if not create if
    if not os.path.exists(f'1stSpanish/3rd_ed_trainingData/{simplified_name}'):
        os.makedirs(f'1stSpanish/3rd_ed_trainingData/{simplified_name}')    
    
    glossDict[simplified_name] = label_dict[text_bbox]

    # Convert the image bbox to a fitz.Rect object
    image_rect = fitz.Rect(*closest_left_bbox)

    # This is the data generated to train the model
    for i in range(1, 11):
        # Define diferent scaling matrices
        mat = fitz.Matrix(i, i)  # This will scale the image to 200%

        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap(clip=image_rect, matrix=mat)

        # Save the image
        pix.save(f'1stSpanish/3rd_ed_trainingData/{simplified_name}/{simplified_name}_small{i}.png')


        # Convert the pixmap to a PIL image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Resize the image
        new_size = (pix.width*10, pix.height*10)  # New size (width, height)
        img_resized = img.resize(new_size)

        # Save the resized image
        img_resized.save(f'1stSpanish/3rd_ed_trainingData/{simplified_name}/{simplified_name}_large{i}.png')



# preview the ref rectangles used to get the data (i used it to define the rectangles)


span_attributes=['size', 'flags', 'font', 'color']

url - (9.0, 4, 'Montserrat-Medium', 15401096) - span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Medium' and span['color'] == 15401096 --> block['bbox']

company name  - (30.0, 20, 'Montserrat-Bold', 0) - span['size'] == 30.0 and span['flags'] == 20 and span['font'] == 'Montserrat-Bold' and span['color'] == 0 --> block['bbox']

product name - (30.0, 20, 'Montserrat-Bold', 15827) - span['size'] == 30.0 and span['flags'] == 20 and span['font'] == 'Montserrat-Bold' and span['color'] == 15827 --> block['bbox']

x0, y0, x1, and y1  of 'scope':
x0 if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block x0
y0  if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block y1

x1 if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block x1
y1 - ??

x0, y0, x1, and y1  of 'destType':
x0 if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block x1
y0  if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block y1
x1 if 'Type of solution' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block x1
y1 - ??

x0, y0, x1, and y1  of 'solType':
x0 if 'Type of solution' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block x1
y0  if 'Scope of smart' in span['text'] and span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696 --> block y1
x1 page width
y1 - ??



# Initial version for getting the html data

In [ ]:
from PIL import Image
# import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors

import IPython.display as display




# draw a rectangle for each position in each page of the pdf
def draw_the_comb_rect(pgList='all', pdfFile=pdfFile):
    """ This function takes:
    - a list of pages (pgList) or 'all' to iterate through all the pages
    - a pdf file (pdfFile)
    - a dictionary with the different attribute combinations and a list of pages where they appear (attr_pos_dict)
    - a dictionary with the different attribute combinations and their positions (text_patterns)
    - the span attributes to be considered (span_attributes): 'all', or a list of
    'size', 'flags', 'font', 'color', 'ascender', 'descender', 'text', 'origin', 'bbox'
    """
    
    doc = fitz.open(pdfFile)

    comp_name = fitz.Rect([0, 0, 0, 0])
    comp_url = fitz.Rect([0, 0, 0, 0])
    sol_name = fitz.Rect([0, 0, 0, 0])
    scope = fitz.Rect([0, 0, 0, 0])
    destType = fitz.Rect([0, 0, 0, 0])
    solType = fitz.Rect([0, 0, 0, 0])
    comp_desc = fitz.Rect([0, 0, 0, 0])
    sol_desc = fitz.Rect([0, 0, 0, 0])


    comp_name_color = (0, 0, 0)
    comp_url_color = (1, 0.5, 1)
    sol_name_color = (1, 0, 0)    
    scope_color = (0, 1, 0)
    destType_color = (0, 0, 1)
    solType_color = (0.5, 0.5, 0.5)
    comp_desc_color = (1, 1, 0)
    sol_desc_color = (0, 1, 1)

    data_holder = {}    

    if pgList == 'all':
        pgList = range(len(doc))

    for i in pgList:
        page = doc[i]

        img_list = []
        #  get the width of the page
        page_width = page.bound().x1
        # get the height of the page
        page_height = page.bound().y1
        for block in page.get_text('dict')['blocks']:

            if block['type'] == 1:
                # page.draw_rect(block['bbox'], color=sol_name_color, width=4)
                img_list.append(block['bbox'])

            if block['type'] == 0:  # Text block
                for line in block['lines']:
                    for span in line['spans']:
                        if span['size'] == 30.0 and span['flags'] == 20 and span['font'] == 'Montserrat-Bold' and span['color'] == 0:
                            comp_name = fitz.Rect(*block['bbox']) # ----> This bbox can be used to get the name of the company
                        elif span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Medium' and span['color'] == 15401096:
                            comp_url = fitz.Rect(*block['bbox']) # ----> This bbox can be used to get the url of the company
                        elif span['size'] > 25 and span['flags'] == 20 and span['font'] == 'Montserrat-Bold' and span['color'] == 15827:
                            sol_name = fitz.Rect(*block['bbox']) # ----> This bbox can be used to get the name of the solution
                        elif span['size'] == 9.0 and span['flags'] == 4 and span['font'] == 'Montserrat-Regular' and span['color'] == 7171696:
                            if 'application' in span['text']:
                                scope_x0 = sol_name[0]
                                scope_x1 = block['bbox'][2]
                                scope_y0 = block['bbox'][1]
                                marker = span['bbox'][3]
                                scope_y1 = marker + 50
                                scope = fitz.Rect([scope_x0, scope_y0, scope_x1, scope_y1]) # ----> This bbox can be used to get the scope of the solution
                                destType_x0 = scope_x1
                                destType_y0 = marker - 5
                                destType_y1 = scope_y1
                                solType_y0 = marker - 5
                                solType_y1 = scope_y1
                            elif 'solution:' in span['text']:
                                destType_x1 = span['bbox'][0] - 50
                                destType = fitz.Rect([destType_x0, destType_y0, destType_x1, destType_y1]) # ----> This bbox can be used to get the type of the solution
                                solType_x0 = destType_x1
                                solType_x1 = page_width - 50
                                solType = fitz.Rect([solType_x0, solType_y0, solType_x1, solType_y1]) # ----> This bbox can be used to get the type of the solution

            

                                 



                comp_desc = fitz.Rect([comp_name[0]-10, comp_name[3], sol_name[0] -10, comp_url[1]]) # ----> This bbox can be used to get the description of the company


                sol_desc = fitz.Rect([sol_name[0]-10, solType[3], solType[2], page_height - 40]) # ----> This bbox can be used to get the description of the solution 

        # -------------------------------------------------------------------------------------------------------------------------------------------------------
        """ # uncomment to check the bboxes for each category
        # draw a rectangle for each position in each page of the pdf
        # draw the copmany name rectangle
        page.draw_rect(comp_name, color=comp_name_color, width=2)
        # draw the company url rectangle
        page.draw_rect(comp_url, color=comp_url_color, width=2)
        # draw the solution name rectangle
        page.draw_rect(sol_name, color=sol_name_color, width=2)
        # draw the scope rectangle
        page.draw_rect(scope, color=scope_color, width=2)
        # draw the destination type rectangle
        page.draw_rect(destType, color=destType_color, width=2)
        # draw the solution type rectangle
        page.draw_rect(solType, color=solType_color, width=2)
        # draw the company description rectangle
        page.draw_rect(comp_desc, color=comp_desc_color, width=2)
        # draw the solution description rectangle
        page.draw_rect(sol_desc, color=sol_desc_color, width=2)


        # Render the page to an image
        pix = page.get_pixmap()

        # Get the image data from the pixmap
        data = pix.samples

        # Create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)

        # Display the image
        display.display(img)# page.saveIncr() """
        # -------------------------------------------------------------------------------------------------------------------------------------------------------

        
        # get the bbox in img_list that is contained in the comp_desc bbox
        for img_bbox in img_list:
            if comp_desc.contains(fitz.Rect(*img_bbox)):
                logo = fitz.Rect(img_bbox)

        # logo = fitz.Rect(img_list[0])

        # if data_holder[i] does not exist create it
        if i not in data_holder.keys():
            data_holder[i] = {'comp_name': comp_name, 'comp_url': comp_url, 'sol_name': sol_name, 'scope': scope, 'destType': destType, 'solType': solType, 'comp_desc': comp_desc, 'sol_desc': sol_desc, 'logo': logo}
        else:
            # if data_holder[i] exists update it
            data_holder[i]['comp_name'] = comp_name
            data_holder[i]['comp_url'] = comp_url
            data_holder[i]['sol_name'] = sol_name
            data_holder[i]['scope'] = scope
            data_holder[i]['destType'] = destType
            data_holder[i]['solType'] = solType
            data_holder[i]['comp_desc'] = comp_desc
            data_holder[i]['sol_desc'] = sol_desc
            data_holder[i]['logo'] = logo

    # close the document
    doc.close()        
    
    return data_holder



        #     #     # in 1st level ordered lists lines start with '#)' or '#.' or 'roman#.' -->I did not find any other pattern for this list level, nor any other levels for ordered lists
        #     #     #  in 1st level unordered lists lines start with '•'
        #     #         # in 2nd level unordered lists lines start with '‒'
        #     #         # in 3rd level unordered lists lines start with '▷'
        #     #     # (span['text'][0] == '•' or (span['text'][0] == ' ' and re.match(r'\d+\.', span['text']) or span['text'][0].isdigit() and span['text'][1] == '.'))
                



        #     #     if comp_name.contains(fitz.Rect(*block['bbox'])):
        #     #         # this is the company  0 name
        #     #         text = [span['text'] for line in block['lines'] for span in line['spans']]
        #     #         company_name = '<h2>' + ' '.join(text).strip() + '</h2>' 
        #     #         print('company_name', company_name)
        #     #     elif comp_url.contains(fitz.Rect(*block['bbox'])):
        #     #         text = [span['text'] for line in block['lines'] for span in line['spans']]
        #     #         company_url = f"<a href=\"https://{' '.join(text).strip()}\" target=\"_blank\">{' '.join(text).strip()}</a>"
        #     #         print('company_url', company_url)
        #     #     elif sol_name.contains(fitz.Rect(*block['bbox'])):
        #     #         text = [span['text'] for line in block['lines'] for span in line['spans'] if span['color'] == 15827]
        #     #         solution_name = '<h1>' + ' '.join(text).strip() + '</h1>'
        #     #         print('solution_name', solution_name)

        #     #     elif comp_desc.intersects(fitz.Rect(*block['bbox'])):
        #     #         text = []
        #     #         previous_line = ''
        #     #         line_text = ''
        #     #         for line in block['lines']:
        #     #             for span in line['spans']:
        #     #                 if rect_in_rect(fitz.Rect(*span['bbox']), comp_desc):
        #     #                     if 'BoldItalic' in span['font']:
        #     #                         line_text += f"<b><i>{span['text']}</i></b>"
        #     #                     elif 'Bold' in span['font']:
        #     #                         line_text += f"<b>{span['text']}</b>"
        #     #                     elif 'Italic' in span['font']:
        #     #                         line_text += f"<i>{span['text']}</i>"
        #     #                     else:
        #     #                         line_text += span['text']
        #     #             if is_ordered_list_item(line_text) or is_unordered_list_item(line_text, 0):
        #     #                 if previous_line:
        #     #                     text.append(previous_line)
        #     #                 previous_line = line_text
        #     #             elif is_unordered_list_item(line_text, 1) or is_unordered_list_item(line_text, 2):
        #     #                 previous_line += '\n' + line_text
        #     #             else:
        #     #                 previous_line += ' ' + line_text
        #     #             line_text = ''
        #     #         # Append the last line if it's not empty
        #     #         if previous_line:
        #     #             text.append(previous_line)

        #     #         # Add a new line after the first period in the end of a line, encountered after the last bullet point
        #     #         text = [line.replace('.', '.\n', 1) if '•' not in line else line for line in text]

        #     #         text_block = '\n'.join(text)
        #     #         company_description = text_to_html(text_block)
        #     #         print('company_description', company_description)


        #     # """ elif comp_desc.contains(fitz.Rect(*block['bbox'])):
        #     #     text = []
        #     #     previous_line = ''
        #     #     line_text = ''
        #     #     for line in block['lines']:
        #     #         for span in line['spans']:
        #     #             if rect_in_rect(fitz.Rect(*span['bbox']), comp_desc):
        #     #                 if 'BoldItalic' in span['font'] and span['text'][0] != '•':
        #     #                     line_text += f"<b><i>{span['text']}</i></b>"
        #     #                 elif 'Bold' in span['font']and span['text'][0] != '•':
        #     #                     line_text += f"<b>{span['text']}</b>"
        #     #                 elif 'Italic' in span['font']and span['text'][0] != '•':
        #     #                     line_text += f"<i>{span['text']}</i>"
        #     #                 else:
        #     #                     line_text += span['text']
        #     #         if line_text.startswith('•'):
        #     #             if previous_line:
        #     #                 text.append(previous_line)
        #     #             previous_line = line_text
        #     #         else:
        #     #             previous_line += ' ' + line_text
        #     #         line_text = ''
        #     #     # Append the last line if it's not empty
        #     #     if previous_line:
        #     #         text.append(previous_line)

        #     #     # Add a new line after the first period in the end of a line, encountered after the last bullet point
        #     #     text = [line.replace('.', '.\n', 1) if '•' not in line else line for line in text]

        #     #     text_block = '\n'.join(text)
        #     #     company_description = text_to_html(text_block)
        #     #     print('company_description', company_description) """
        #     # # elif sol_desc0.contains(fitz.Rect(*block['bbox'])):
        #     # #     spanish_text = []
        #     # #     previous_line = ''
        #     # #     line_text = ''
        #     # #     for line in block['lines']:
        #     # #         # line_text = ' '.join(span['text'] for span in line['spans'])
        #     # #         for span in line['spans']:
        #     # #             if rect_in_rect(fitz.Rect(*span['bbox']), sol_desc0):
        #     # #                 if span['font'] == 'MyriadPro-Semibold' or span['font'] == 'MyriadPro-Bold' and span['text'][0] != '•':
        #     # #                     line_text += f"<b>{span['text']}</b>"
        #     # #                 elif span['font'] ==  'MyriadPro-SemiboldIt' or span['font'] == 'MyriadPro-BoldIt' and span['text'][0] != '•':
        #     # #                     line_text += f"<b><i>{span['text']}</i></b>"
        #     # #                 else:
        #     # #                     line_text += span['text']
        #     # #         if line_text.startswith('•'):
        #     # #             if previous_line:
        #     # #                 spanish_text.append(previous_line)
        #     # #             previous_line = line_text
        #     # #         else:
        #     # #             previous_line += ' ' + line_text
        #     # #         line_text = ''
        #     # #     # Append the last line if it's not empty
        #     # #     if previous_line:
        #     # #         spanish_text.append(previous_line)

        #     # #     # Add a new line after the first period in the end of a line, encountered after the last bullet point
        #     # #     spanish_text = [line.replace('.', '.\n', 1) if '•' not in line else line for line in spanish_text]

        #     # #     spanish_text_block = '\n'.join(spanish_text)
        #     # #     solution0_description = text_to_html(spanish_text_block)
        #     # #     solution0_description = translate(solution0_description)
        #     # #     print('solution0_description', solution0_description)
        #     # # elif sol_target0.contains(fitz.Rect(*block['bbox'])):
        #     # #     spanish_text = [' '.join(span['text'] for span in line['spans']) for line in block['lines']]

        #     # #     spanish_text = translate(' '.join(spanish_text))
        #     # #     spanish_text = spanish_text.split(',')
        #     # #     text = '<p>Target audience</p><ul>'
        #     # #     for line in spanish_text:
        #     # #         text += f'<li>{line}</li>'
        #     # #     solution0_target = text + '</ul>'
        #     # #     print('------->solution0_target', solution0_target)
















# draw_the_comb_rect(pgList=range(9,363), pdfFile=pdfFile)

In [ ]:
import IPython.display as display
from fastbook import *

learn_inf = load_learner('1stSpanish/1stSpanish_3rdEd_vision.pkl')

def get_area_drawings(page, frontier, area_threshold=1000):
    
    # print('frontier:', frontier)
    # 1=====================================================================================================
    drawings = page.get_drawings()
    rect_list = []
    # page.draw_rect(frontier, color=(0, 0, 1), width=2)
    for path in drawings:            
        rect = path['rect']
        # print(rect, 'area: ',(rect[2]-rect[0]) *( rect[3]-rect[1]))
        area = (rect[2]-rect[0])*( rect[3]-rect[1])
        if frontier.contains(rect) and area > area_threshold:
            rect_list.append(rect)
    #---------------------------------------------------------
    # Create a new list to store the rectangles that are not contained within any other rectangle
    new_rect_list = []

    for i, rect in enumerate(rect_list):
        # Assume the current rectangle is not contained within any other rectangle
        contained = False
        for j, other_rect in enumerate(rect_list):
            if i != j and rect != other_rect and other_rect.contains(rect):
                # If the current rectangle is contained within another rectangle, set the flag to True
                contained = True
                break
        if not contained:
            # If the current rectangle is not contained within any other rectangle, add it to the new list
            new_rect_list.append(rect)

    # Replace the old list with the new list
    rect_list = new_rect_list

    # remove duplicates in rect_list
    rect_list = list(dict.fromkeys(rect_list))
    # /1====================================================================================================   
    
    return rect_list


def get_infografic_text_data(page, frontier, area_threshold=1000):


    destTypes = get_area_drawings(page, frontier, area_threshold=1000)

    destType_list = []

    # print('I got HEre to get_infografic_text_data')

    for destType_bbox in destTypes:
        page.draw_rect(destType_bbox, color=(1, 0, 0), width=2)

        # Define a scaling matrix
        mat = fitz.Matrix(10, 10)

        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap(clip=destType_bbox, matrix=mat)
        data = pix.samples
        # create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)

        # # # Display the image
        # display(img)

    # Uses the label_converter dict to convert the label to the original text in the glossary  ----> THIS IS THE DICTIONA TO CONVERT THE LABELED TEXT FROM THE MODEL OUTPUT TO THE ORIGINAL TEXT
        pred = learn_inf.predict(img)
        pred_text = glossDict[pred[0]]#label_converter[pred[0]]
        # print(learn_inf.predict(img)[0])
        # print(learn_inf.predict(img))
        destType_list.append(pred_text)
    
    # Append 'destination type:' to the first element of the list
    destType_list[0] = 'destination type: ' + "\n• " + destType_list[0]

    text_block_data = "\n• ".join(destType_list)

    destination_type = text_to_html6(text_block_data)

    # print(i,'destination_type', destination_type)

    return destination_type

def get_infografic_list_data(page, frontier, area_threshold=1000):


    destTypes = get_area_drawings(page, frontier, area_threshold=1000)

    destType_list = []

    # print('I got HEre to get_infografic_text_data')

    for destType_bbox in destTypes:
        page.draw_rect(destType_bbox, color=(1, 0, 0), width=2)

        # Define a scaling matrix
        mat = fitz.Matrix(10, 10)

        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap(clip=destType_bbox, matrix=mat)
        data = pix.samples
        # create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)

        # # # Display the image
        # display(img)

    # Uses the label_converter dict to convert the label to the original text in the glossary  ----> THIS IS THE DICTIONA TO CONVERT THE LABELED TEXT FROM THE MODEL OUTPUT TO THE ORIGINAL TEXT
        pred = learn_inf.predict(img)
        pred_text = glossDict[pred[0]]#label_converter[pred[0]]
        # print(learn_inf.predict(img)[0])
        # print(learn_inf.predict(img))
        destType_list.append(pred_text)
    
    return destType_list
    


# check the draw_the_comb_rect()

In [ ]:
data_places_blocks = {}
pgList=range(9,363)
data_places = draw_the_comb_rect(pgList, pdfFile)
doc = fitz.open(pdfFile)
#see all pages 
""" for i, value in data_places.items():
    page = doc[i]
    blockDict = page.get_text('dict')['blocks']
    for key, items in value.items():
        print('key', key, 'items', items)

        # draw the rectangle on the page
        page.draw_rect(items, color=(0, 0, 1), width=2)

        blockDict = get_dataDict(i, pdfFile, items)
        print('scope blockDict len', len(blockDict))
        scope =[]                
        for block in blockDict:
            page.draw_rect(block['bbox'], color=(1, 0, 0), width=2)


    # Render the page to an image
    pix = page.get_pixmap()

    # Get the image data from the pixmap
    data = pix.samples

    # Create a PIL image from the data
    img = Image.frombytes("RGB", [pix.width, pix.height], data)

    # Display the image
    display.display(img)# page.saveIncr() """
    
# page_nr = 348
# page = doc[page_nr]
# for key, items in data_places[page_nr].items():
#     if key == 'destType':
#         print(get_infografic_list_data(page, items, area_threshold=1000))
#         print(get_area_drawings(page, items, area_threshold=1000))
#         for rect in get_area_drawings(page, items, area_threshold=1000):
#             page.draw_rect(rect, color=(1, 0, 0), width=2)
#     # print('key', key, 'items', items)

#     # draw the rectangle on the page
#     page.draw_rect(items, color=(0, 0, 1), width=2)

#     blockDict = get_dataDict(page_nr, pdfFile, items)
#     # print('scope blockDict len', len(blockDict))
#     # scope =[]                
#     # for block in blockDict:
#         # page.draw_rect(block['bbox'], color=(1, 0, 0), width=2)
#         # if key == 'sol_name':
#         #     print(block)


# # Render the page to an image
# pix = page.get_pixmap()

# # Get the image data from the pixmap
# data = pix.samples

# # Create a PIL iget_dataDictmage from the data
# img = Image.frombytes("RGB", [pix.width, pix.height], data)

# # Display the image
# display.display(img)# page.saveIncr()


In [ ]:
import re

# from bs4 import BeautifulSoup

import base64


# def remove_redundant_tags(html):
#     soup = BeautifulSoup(html, 'html.parser')

#     for tag in soup.find_all():
#         while tag.find_next_sibling() and tag.find_next_sibling().name == tag.name:
#             tag.string = f"{tag.string} {tag.find_next_sibling().string}"
#             tag.find_next_sibling().decompose()

#     return str(soup)

def is_ordered_list_item(text):
    return re.match(r'^\d+\)\s|^\d+\.\s|^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s|^(<[^<]+?>)*\d+\)\s|^(<[^<]+?>)*\d+\.\s|^(<[^<]+?>)*(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s', text, re.IGNORECASE) is not None

def is_unordered_list_item(text, level):
    markers = ['•', '‒', '▷']
    return any(re.match(f'^(<[^<]+?>)*{marker}|^{marker}', text) for marker in markers[level])

def rect_in_rect(rect1, rect2):
    """Check if rect1 is inside rect2."""
    return rect2.contains(rect1)

def roman_to_int(s):
    roman_numerals = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
    int_val = 0
    for i in range(len(s)):
        if i > 0 and roman_numerals[s[i]] > roman_numerals[s[i - 1]]:
            int_val += roman_numerals[s[i]] - 2 * roman_numerals[s[i - 1]]
        else:
            int_val += roman_numerals[s[i]]
    return int_val

def unordered_lists_summarizer(text):
    lines = text.split('\n')
    unordered_list_markers = []
    unordered_list_lines = []

    for i, line in enumerate(lines):
        stripped_line = line.strip()
        if any(is_unordered_list_item(stripped_line, level) for level in range(3)):
            unordered_list_markers.append(stripped_line[0])
            unordered_list_lines.append(i)
        elif is_ordered_list_item(stripped_line):
            unordered_list_markers.append(stripped_line[0])
            unordered_list_lines.append(i)
        elif stripped_line[:4] == '<li>' or stripped_line[:4] == '<ul>' or stripped_line[:4] == '<ol>':
            unordered_list_markers.append(stripped_line[:4])
            unordered_list_lines.append(i)
        elif stripped_line[:5] == '</li>' or stripped_line[:5] == '</ul>' or stripped_line[:5] == '</ol>':
            unordered_list_markers.append(stripped_line[:5])
            unordered_list_lines.append(i)
        else:
            unordered_list_markers.append('P')
            unordered_list_lines.append(i)
    
    return unordered_list_markers, unordered_list_lines


# -----------------------------------------------------------------------------
def get_list_number(line):
    match = re.match(r'\s*(\d+)[.)]', line)
    if match:
        return int(match.group(1))
    else:
        return None

def ordered_lists_summarizer(text):
    lines = text.split('\n')
    ordered_list_nrs = []
    ordered_list_lines = []

    for i, line in enumerate(lines):
        stripped_line = line.strip()
        if is_ordered_list_item(stripped_line):
            ordered_list_nrs.append(get_list_number(stripped_line))
            ordered_list_lines.append(i)
    return ordered_list_nrs, ordered_list_lines

from itertools import groupby
from operator import itemgetter

def find_sequences(ordered_list_nrs):
    for k, g in groupby(enumerate(ordered_list_nrs), lambda i_x:i_x[0]-i_x[1]):
        sequence = list(map(itemgetter(1), g))
        if sequence[0] == 1:  # Only consider sequences starting from 1
            yield sequence

def find_all_sequences(ordered_list_nrs):
    for k, g in groupby(enumerate(ordered_list_nrs), lambda i_x:i_x[0]-i_x[1]):
        sequence = list(map(itemgetter(1), g))
        yield sequence

""" def handle_ordered_lists(text):
    lines = text.split('\n')
    ordered_list_nrs, ordered_list_lines = ordered_lists_summarizer(text)

    if ordered_list_nrs:

        while ordered_list_nrs:
            sequences_found = False
            # Find sequences of ordered list numbers starting from 1
            for sequence in find_sequences(ordered_list_nrs):
                sequences_found = True
                # Find the start and end indices of the sequence in the ordered_list_nrs list
                start_index = ordered_list_nrs.index(sequence[0])
                end_index = start_index + len(sequence) - 1

                # Find the corresponding line numbers
                start_line = ordered_list_lines[start_index]
                end_line = ordered_list_lines[end_index]
                print('start_line', start_line)
                print('end_line', end_line)

                # Add the <ol> and </ol> tags
                for i, line in enumerate(lines):
                    stripped_line = line.strip()
                    if is_ordered_list_item(stripped_line) and i == start_line:
                        lines[i] = '<ol>\n' + stripped_line
                    elif is_ordered_list_item(stripped_line) and i == end_line:
                        lines[i] = stripped_line + '\n</ol>'
                

                
                # remove the the list items in 'ordered_list_nrs', 'ordered_list_lines' from 'start_index' to 'end_index'
                ordered_list_nrs = ordered_list_nrs[:start_index] + ordered_list_nrs[end_index + 1:]
                ordered_list_lines = ordered_list_lines[:start_index] + ordered_list_lines[end_index + 1:]
            if not sequences_found:
                print('no sequences found!!!!!!!!!!!!!')
                break

    ol_text = '\n'.join(lines)
    return ol_text """
def check_tuples(tuples):
    values = [t[0] for t in tuples]
    all_same = all(t[0] == t[1] for t in tuples)
    all_sequence = all(values[i] - values[i-1] == 1 for i in range(1, len(values)))
    return values, all_same, all_sequence

def check_tuples_rev(tuples):
    values = [t[0] for t in tuples]
    all_same = all(t[0] == t[1] for t in tuples)
    all_sequence = all(values[i] - values[i-1] == -1 for i in range(1, len(values)))
    return values, all_same, all_sequence

def find_sublist(lst, sublist):
    length = len(sublist)
    for i in range(len(lst)):
        if lst[i:i+length] == sublist:
            return i, i+length-1  # -1 to get the last index in the sublist
    return -1, -1  # Return -1, -1 if the sublist is not found


def get_last_consecutive_index_of_X_after_MAX(lista, X, max_index=None):
    if max_index is None:
        max_index = lista.index(max(lista))
    # print('max_index', max_index)
    count = 0
    for value in lista[max_index + 1:]:
        if X[0] <= value >= X[1]:
            count += 1
        else:
            break
    return max_index + count

import random
def handle_ordered_lists(text):
    print('this is the text', text)

    lines = text.split('\n')

    # print('lines[:2]')
    # print([line.strip()[:2] for line in lines])

    summary_list = []
    summary_line_indexes = []
    random_number_list = []
    for line in lines:        
        if any(is_unordered_list_item(line.strip(), level) for level in range(3)):
            # append a random negative number from [-100 to 0] to the summary_list, that is not in, nor consecutive to any number in the random_number_list
            while True:
                random_number = random.randint(-100, -1)
                if random_number not in random_number_list and all(abs(random_number - x) > 1 for x in random_number_list):
                    random_number_list.append(random_number)
                    break
            summary_list.append(random_number)
            summary_line_indexes.append(str(lines.index(line)))
        elif is_ordered_list_item(line.strip()):
                summary_list.append(int(get_list_number(line.strip())))
                summary_line_indexes.append(lines.index(line))
        else:
            while True:
                random_number = random.randint(-1000, -900)
                if random_number not in random_number_list and all(abs(random_number - x) > 1 for x in random_number_list):
                    random_number_list.append(random_number)
                    break
            summary_list.append(random_number)
            summary_line_indexes.append(None)
    # print('summary_list', summary_list)
    # print('summary_line_indexes', summary_line_indexes)

    if 1 not in summary_list:
        pass
    else:
        # print('summary_list', summary_list)
        sequences = list(find_all_sequences(summary_list))
        # print('sequences', sequences)
        # sequence_indexes = [find_sublist(summary_list, sequence) for sequence in sequences]
        # print('sequence_indexes', sequence_indexes)

        simplified_summary_line_indexes = [element for element in summary_line_indexes if element is not None and isinstance(element, int)]

        # print('simplified_summary_line_indexes', simplified_summary_line_indexes)
        # sequence_indexes = list(find_all_sequences(simplified_summary_line_indexes))
        # print('sequence_indexes', sequence_indexes)

        # print(len(sequences) == len(sequence_indexes))
        # lister = []
        # for element in summary_line_indexes:
        #     if element is not None and isinstance(element, int):
                
        #         lister.append(element)
        #     else:
        #         lister.append



        # if text only has a <ol> list (sequence starting from 1)
        if summary_list.count(1) == 1:
            lines[summary_line_indexes[summary_list.index(1)]] = '<ol>\n' + lines[summary_line_indexes[summary_list.index(1)]].strip()

            # if the max number in summary_list is followed by int in -100, -1 range, find the index of the last consecutive -1 and add </ol> to the line with that index
            # if the list item ends with ':' add a line
            if lines[get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1))].strip()[-1] == ':':
                lines[get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1)) + 1] = lines[get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1)) + 1].strip() + '\n</ol>'
            else:
                lines[get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1))] = lines[get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1))].strip() + '\n</ol>'
            


            # print(get_last_consecutive_index_of_X_after_MAX(summary_list, (-100, -1)))

        else: # more than one <ol> list
            #  if lists are not nested. the count of lits with '1' equals the count of lists with positive numbers
            positive_sublists = [sublist for sublist in sequences if any(item > 0 for item in sublist)]
            # print('positive_sublists', positive_sublists)
            positive_count = len(positive_sublists)
            # print('positive_count', positive_count)
            # print('summary_list.count(1)', summary_list.count(1))

            # print(type(positive_sublists))
            # print(type(positive_sublists[0]))
            # print(type(positive_sublists[0][0]))

            simp = []
            for subl in positive_sublists:
                listy = []
                counter = 0
                for i in range(0,len(subl)):
                    counter += 1
                    listy.append(simplified_summary_line_indexes[i])

                for i in range(0,counter):                   
                    simplified_summary_line_indexes.pop(0)
                simp.append(listy)
                
            # print('simp', simp)

            sequence_indexes = summary_line_indexes

            # for i, element in enumerate(sequence_indexes):
            #     if not isinstance(element, int):
            #         sequence_indexes[i] = (i, i)

            # print('sequence_indexes', sequence_indexes)
            # print('len(sequence_indexes)', len(sequence_indexes))
                    

            
   
            for sublist in simp:
                start, end = find_sublist(summary_line_indexes, sublist)
                # for i in range(start, end):
                #     if i == start:
                if len(sublist) == 1:
                    sublist = [sublist[0], sublist[0]]
                sequence_indexes[start] = tuple(sublist)
            # print('sequence_indexes', sequence_indexes)
            # print('len(sequence_indexes)', len(sequence_indexes))

            # remove all elements that are int in sequence_indexes
            sequence_indexes = [element for element in sequence_indexes if not isinstance(element, int)]

            # print('sequence_indexes', sequence_indexes)
            # print('len(sequence_indexes)', len(sequence_indexes))
            
            if summary_list.count(1) == positive_count:
                #  lists are not nested
                #  Get the indexes of the positive sublists
                for positive_sublist in positive_sublists:
                    # print('is this here?')
                    # print('positive_sublist', positive_sublist)
                    # print(sequence_indexes[sequences.index(positive_sublist)])
                    #  add the <ol> tag to the line with the index of the first item in the sublist
                    lines[sequence_indexes[sequences.index(positive_sublist)][0]] = '<ol>\n' + lines[sequence_indexes[sequences.index(positive_sublist)][0]].strip()

                    #  add the </ol> tag to the line with the index of the last item in the sublist
                    # if the list item ends with ':' add a line
                    # if the max number in summary_list is followed by int in -100, -1 range, find the index of the last consecutive -1 and add </ol> to the line with that index and the the last item in the sublist ends with ':' add a line

                    if lines[sequence_indexes[sequences.index(positive_sublist)][0]][-1].strip() == ':':
                        if lines[get_last_consecutive_index_of_X_after_MAX(summary_list[sequence_indexes[sequences.index(positive_sublist)][-1]:], (-100, -1))] == ':':
                            lines[get_last_consecutive_index_of_X_after_MAX(summary_list[sequence_indexes[sequences.index(positive_sublist)][-1]:], (-100, -1)) + 1] = lines[get_last_consecutive_index_of_X_after_MAX(summary_list[sequence_indexes[sequences.index(positive_sublist)][1]:], (-100, -1)) + 1].strip() + '\n</ol>'
                        else:
                            lines[get_last_consecutive_index_of_X_after_MAX(summary_list[sequence_indexes[sequences.index(positive_sublist)][-1]:], (-100, -1))] = lines[get_last_consecutive_index_of_X_after_MAX(summary_list[sequence_indexes[sequences.index(positive_sublist)][1]:], (-100, -1))].strip() + '\n</ol>'
                    else:
                        # print('yetrgs',  lines[sequence_indexes[sequences.index(positive_sublist)][1]-1].strip())
                        lines[sequence_indexes[sequences.index(positive_sublist)][-1]] = lines[sequence_indexes[sequences.index(positive_sublist)][-1]].strip() + '\n</ol>'
            else:
                #  lists are nested
                # print(positive_sublists)
                # remove all elements that are text or None in sequence_indexes
                positive_sublists_indexes = [element for element in sequence_indexes if isinstance(element, tuple)]
                # unpak the tuples to lists
                positive_sublists_indexes = [list(element) for element in positive_sublists_indexes]
                # print('positive_sublists_indexes', positive_sublists_indexes)
                # print('len(positive_sublists_indexes)', len(positive_sublists_indexes))
                #  remove all all lists in sequences list that have negative numbers
                positive_sublists_numbers = [sequence for sequence in sequences if any(item > 0 for item in sequence)]
                # print('positive_sublists_numbers', positive_sublists_numbers)
                # print('len(positive_sublists_numbers)', len(positive_sublists_numbers))


                #  If the any of the sublists in the positive_sublists_indexes is not sequential to any of the other sublists, 
                # append <ol> to the line with the index of the first item in the sublist and </ol> to the line with the index of the last item in the sublist, and pop the sublist from the positive_sublists_indexes list                
                for i, element in enumerate(positive_sublists_indexes):
                    if element == positive_sublists_indexes[-1]:
                        break
                    if element[-1] != positive_sublists_indexes[i+1][0] - 1:
                        lines[element[0]] = '<ol>\n' + lines[element[0]].strip()
                        lines[element[-1]] = lines[element[-1]].strip() + '\n</ol>'
                        positive_sublists_indexes.pop(i)
                        positive_sublists_numbers.pop(i)

                # print('positive_sublists_indexes', positive_sublists_indexes)
                # print('len(positive_sublists_indexes)', len(positive_sublists_indexes))
                
                # now we are only dealing with sequential sublists

                # reverse the positive_sublists_indexes list
                rev_positive_sublists_indexes = list(reversed(positive_sublists_indexes))
                # print('rev_positive_sublists_indexes', rev_positive_sublists_indexes)
                # print('len(rev_positive_sublists_indexes)', len(rev_positive_sublists_indexes))

                # reverse the positive_sublists_numbers list
                rev_positive_sublists_numbers = list(reversed(positive_sublists_numbers))   
                # print('rev_positive_sublists_numbers', rev_positive_sublists_numbers)
                # print('len(rev_positive_sublists_numbers)', len(rev_positive_sublists_numbers))

                # for i, element in enumerate(rev_positive_sublists_indexes):
                #     for j in element:
                        # print(lines[j])


                #  add the <ol> tag to the line with the index of the first item in the sublist                
                opened_ends_list = []# here we will append the indexes, and the values (index, value) of the opened ends of the sublists
                mother_list = [(rev_positive_sublists_indexes[0][0],positive_sublists_numbers[0][0])]

                counter = 0
                while len(rev_positive_sublists_numbers) > 1:
                    counter += 1
                    for i, element in enumerate(rev_positive_sublists_numbers):
                        if i == 0 and element[0] == 1:
                            # print(0, 'element[0]', element[0], 'i', i)
                            lines[rev_positive_sublists_indexes[i][0]] = '<ol>\n' + lines[rev_positive_sublists_indexes[i][0]].strip()
                            lines[rev_positive_sublists_indexes[i][-1]] = lines[rev_positive_sublists_indexes[i][-1]].strip() + '\n</ol>'
                            rev_positive_sublists_indexes.pop(i)
                            rev_positive_sublists_numbers.pop(i)
                            # print(0, lines[rev_positive_sublists_indexes[i][0]],'\n...\n' , lines[rev_positive_sublists_indexes[i][-1]])
                            # print(rev_positive_sublists_numbers[i][0])
                        elif i == 0 and element[0] != 1:
                            # print(1, 'element[0]', element[0], 'i', i)
                            # we are missing the start of this sublist
                            # find the index of the last opened end
                            opened_ends_list.append((i, element[-1]))
                            # lines[rev_positive_sublists_indexes[i][-1]] = lines[rev_positive_sublists_indexes[i][-1]].strip() + '\n<ol>'
                            # print(1, lines[rev_positive_sublists_indexes[i][0]],'\n...\n' , lines[rev_positive_sublists_indexes[i][-1]])
                        elif i != 0 and element[0] == 1:
                            # print(2, 'element[0]', element[0], 'i', i)
                            lines[rev_positive_sublists_indexes[i][0]] = '<ol>\n' + lines[rev_positive_sublists_indexes[i][0]].strip()
                            if element[-1] + 1 not in opened_ends_list:
                                lines[rev_positive_sublists_indexes[i][-1]] = lines[rev_positive_sublists_indexes[i][-1]].strip() + '\n</ol>'
                                rev_positive_sublists_indexes.pop(i)
                                rev_positive_sublists_numbers.pop(i)
                                break
                    #         print(2, lines[rev_positive_sublists_indexes[i][0]],'\n...\n' , lines[rev_positive_sublists_indexes[i][-1]])
                            
                    # print('====================================================================================================')    
                    # print('rev_positive_sublists_indexes', rev_positive_sublists_indexes)
                    # print('rev_positive_sublists_numbers', rev_positive_sublists_numbers)
                    # print('----------------------------------------------------------------------------------------------------')

                    # if rev_positive_sublists_numbers[0] == rev_positive_sublists_numbers[1] + 1 merge the two sublists
                    if len(rev_positive_sublists_numbers) > 2 and rev_positive_sublists_numbers[0][-1] == rev_positive_sublists_numbers[1][0] + 1 and rev_positive_sublists_numbers[0][-1] != 1:
                        rev_positive_sublists_indexes[0] = [rev_positive_sublists_indexes[1][-1], rev_positive_sublists_indexes[0][0]]
                        rev_positive_sublists_numbers[0] = [rev_positive_sublists_numbers[1][-1], rev_positive_sublists_numbers[0][0]]
                        rev_positive_sublists_indexes.pop(1)
                        rev_positive_sublists_numbers.pop(1)
                    else:
                        pass

                    if len(rev_positive_sublists_numbers) == 2:# and rev_positive_sublists_numbers[0][-1] == rev_positive_sublists_numbers[1][0] + 1:
                        lines[rev_positive_sublists_indexes[1][0]] = '<ol>\n' + lines[rev_positive_sublists_indexes[1][-1]].strip()

                        # lines[mother_list[0][0]] = lines[opened_ends_list[0][0]].strip() + '\n</ol>'
                        lines[rev_positive_sublists_indexes[0][-1] + 1] = lines[rev_positive_sublists_indexes[0][-1] + 1].strip() + '\n</ol>'

                        rev_positive_sublists_indexes.pop(1)
                        rev_positive_sublists_numbers.pop(1)                        
                        rev_positive_sublists_indexes.pop(0)
                        rev_positive_sublists_numbers.pop(0)


                    # print('rev_positive_sublists_indexes', rev_positive_sublists_indexes)
                    # print('rev_positive_sublists_numbers', rev_positive_sublists_numbers)
                    # print('====================================================================================================')

                    if counter == 10:
                        break


                # # reverse the positive_sublists_indexes list
                # print('rev_positive_sublists_indexes', rev_positive_sublists_indexes)
                # print('len(rev_positive_sublists_indexes)', len(rev_positive_sublists_indexes))

                # # reverse the positive_sublists_numbers list 
                # print('rev_positive_sublists_numbers', rev_positive_sublists_numbers)
                # print('len(rev_positive_sublists_numbers)', len(rev_positive_sublists_numbers))


    ol_text = '\n'.join(lines)
    return ol_text

def get_last_unordered_marker_in_sequence(text, my_marker, range_start, range_end):
    # Define the list markers for each level
    list_markers = ['•', '‒', '▷']
    unordered_list_markers, unordered_list_lines = unordered_lists_summarizer(text)
    nested_list_markers = []
    found_marker = None
    last_sequential_symbol = None
    last_sequential_index = None
    for i in range(range_start, range_end):
        if unordered_list_markers[i] in list_markers:
            if unordered_list_markers[i] != my_marker and unordered_list_markers[i] in list_markers:
                nested_list_markers.append(i)
                found_marker = unordered_list_markers[i]

            last_sequential_symbol = unordered_list_markers[i]
            last_sequential_index = i
        elif unordered_list_markers[i] not in list_markers:
            break
    return unordered_list_markers, unordered_list_lines, last_sequential_symbol , last_sequential_index, found_marker, nested_list_markers    
    """ if not nested_list_markers:
        return unordered_list_markers, unordered_list_lines, last_unordered_marker, last_unordered_index, found_marker, nested_list_markers
    else: # found nested unordered list markers in the unordered list
        return unordered_list_markers, unordered_list_lines, last_unordered_marker, last_unordered_index, found_marker, nested_list_markers """

def merge_sequential_tuples(tuples):
    merged = []
    current_start, current_end = tuples[0]

    for start, end in tuples[1:]:
        if start - 1 <= current_end:  # If the start of the next tuple is <= end of the current tuple + 1
            current_end = max(current_end, end)  # Extend the current tuple
        else:  # If the start of the next tuple is > end of the current tuple + 1
            merged.append((current_start, current_end))  # Add the current tuple to the merged list
            current_start, current_end = start, end  # Start a new tuple

    merged.append((current_start, current_end))  # Add the last tuple

    return merged

import sys
from IPython.display import display, Markdown

def find_subranges(nested_list_markers1, unordered_list_markers, text, markerlist=['•', '‒', '▷'], level=1):
    # Define the list markers for each level
    subranges = []
    start_index = nested_list_markers1[0]
    current_value = unordered_list_markers[start_index]

    for i in range(nested_list_markers1[0], nested_list_markers1[-1]):
        if unordered_list_markers[i] != current_value:
            subranges.append((start_index, i -1))  # Add the subrange (start_index, end_index)
            start_index = i
            current_value = unordered_list_markers[i]

    subranges.append((start_index, nested_list_markers1[-1]))  # Add the last subrange

    merge = merge_sequential_tuples(subranges)



    # now we nedd to analyse the elements within each merge groups:
    #  if elements in a sequencial range have equal or higher index of list_markers,
    #  then we create a range starting from the first element of the range and ending with the last element of the range in merge
    #  if elements in a sequencial range have lower index of list_markers,we ignore this elements
    final_subranges = []
    lines = text.split('\n')
    base_marker_index = level # 0 is the index for 1st level unordered list marker
    for large_range in merge:
        large_range_start, large_range_end = large_range
        for subrange in subranges:
            start, end = subrange
            if lines[start].strip()[0] == lines[end].strip()[0]:
                marker = lines[start].strip()[0]
            else:
                #  display an alert window and stop the program
                display(Markdown('**Error: {} != {} **'.format(lines[start].strip()[0], lines[end].strip()[0])))
                sys.exit()
            # get the index of the marker in markerlist
            marker_index = markerlist.index(marker)
            if marker_index >= base_marker_index:
                avrg_point = (start + end) // 2
                if large_range[0] <= avrg_point and large_range[1] >= avrg_point:
                    # check if the avrg_point > range[0] and < range[1]
                    final_subranges.append((start, end))#large_range_end))
                    """ This could be usefful for analysing nested lists starting from the first level """
                    # check if min(subranges[i][0]) >= large_range_start and max(subranges[i][1]) <= large_range_end
                    # if all([subrange for subrange in subranges if subrange[0] >= large_range_start and subrange[1] <= large_range_end]):
                    #     print('Here i am lost in the middle with you')
                    #     final_subranges.append((start, large_range_end))
                    # else:
                    #     final_subranges.append((start, end))#large_range_end))
    return final_subranges

""" def find_html_encapsulated_lines(text, tags):
    lines = text.split('\n')
    not_html_encapsulated_lines = []
    not_html_encapsulated_indexes = []
    tags = [(f'<{tag}>',f'</{tag}>') for tag in tags]    
    capsuleCorp = []
    last_found = 'end_tag'
    last_i = []
    for tag in tags:
        start_tag, end_tag = tag
        start_at = 0
        for i, line in enumerate(lines):
            stripped_line = line.strip()
            if start_tag in stripped_line:
                if last_found == 'start_tag':
                    last_i.append(i)
                    last_found = 'nested_start_tag'
                elif last_found == 'end_tag':
                    start_at = i
                    last_found = 'start_tag'
                elif last_found == 'nested_start_tag':
                    last_i.append(i)
                    last_found = 'nested_start_tag'

            elif end_tag in stripped_line:
                if last_found == 'start_tag':
                    capsuleCorp.append((start_at, i))
                    last_found = 'end_tag'
                elif last_found == 'nested_start_tag':
                    capsuleCorp.append((last_i[-1], i))
                    # remove last_i[-1] from last_i
                    last_i.pop()
                    if last_i:
                        last_found = 'nested_start_tag'
                    else:
                        last_found = 'end_tag'

    for i, line in enumerate(lines):
        if not any(start <= i <= end for start, end in capsuleCorp):
            not_html_encapsulated_lines.append(line)
            not_html_encapsulated_indexes.append(i)
    print('not_html_encapsulated_lines', not_html_encapsulated_lines)
    print('not_html_encapsulated_indexes', not_html_encapsulated_indexes)
    print('capsuleCorp', capsuleCorp)
    return not_html_encapsulated_indexes, not_html_encapsulated_lines, capsuleCorp """

def find_html_encapsulated_lines(text, tags):
    lines = text.split('\n')
    not_html_encapsulated_lines = []
    not_html_encapsulated_indexes = []
    tags = [(f'<{tag}>',f'</{tag}>') for tag in tags]    
    capsuleCorp = []
    tag_stack = []
    for i, line in enumerate(lines):
        stripped_line = line.strip()
        for start_tag, end_tag in tags:
            if start_tag in stripped_line:
                tag_stack.append((start_tag, i))
            elif end_tag in stripped_line:
                if tag_stack and tag_stack[-1][0] == start_tag:
                    start_at = tag_stack.pop()[1]
                    capsuleCorp.append((start_at, i))
    for i, line in enumerate(lines):
        if not any(start <= i <= end for start, end in capsuleCorp):
            not_html_encapsulated_lines.append(line)
            not_html_encapsulated_indexes.append(i)

    return not_html_encapsulated_indexes, not_html_encapsulated_lines, capsuleCorp

def text_to_html6(text):
    # Define the list markers for each level
    list_markers = ['•', '‒', '▷']
    """ unordered_list_markers, unordered_list_lines = unordered_lists_summarizer(text) """
    lines = text.split('\n')
    my_marker = None
    range_start = 0
    range_end = len(lines)
    unordered_list_markers, unordered_list_lines, last_sequential_symbol , last_sequential_index, found_marker, nested_list_markers  = get_last_unordered_marker_in_sequence(text, my_marker, range_start, range_end)

    for i, line in enumerate(lines):
        stripped_line = line.strip()
        if is_ordered_list_item(stripped_line):
            if (i + 1 in unordered_list_lines and unordered_list_markers[i + 1] in list_markers):
                # find the last sequential simbol starting from i + 1 in unordered_list_markers that is in list_markers
                # and find the index of the last sequential simbol starting from i + 1 in unordered_list_markers that is in list_markers

                my_marker = lines[i+1].strip()[0]
                range_start = i + 1
                # print('text', text)
                # print('my_marker', my_marker)
                # print('range_start', range_start)
                # print('range_end', range_end)
                # print('unordered_list_markers', unordered_list_markers)
                # print('unordered_list_lines', unordered_list_lines)
                _, _, last_sequential_symbol , last_sequential_index1, found_marker1, nested_list_markers1  = get_last_unordered_marker_in_sequence(text, my_marker, range_start, range_end)
                # print('last_sequential_symbol', last_sequential_symbol)
                # print('last_sequential_index1', last_sequential_index1)
                # print('found_marker1', found_marker1)
                # print('nested_list_markers', nested_list_markers)

                lines[i + 1] = '<ul>\n' + lines[i + 1]

                if lines[last_sequential_index1][:7] == '\n</ol>':
                    # insert the '\n</ul>' before '\n</ol>
                    lines[last_sequential_index1] = lines[last_sequential_index1].replace('\n</ol>','') + '\n</ul>\n<ol>'
                else:
                    lines[last_sequential_index1] = lines[last_sequential_index1] + '\n</ul>'
                # lines[last_sequential_index1] = lines[last_sequential_index1] + '\n</ul>'
                
                if found_marker1:
                    # check if all markers in nested_list_markers1 are the same
                    nested_sub = find_subranges(nested_list_markers1, unordered_list_markers, text, markerlist=['•', '‒', '▷'], level=1)

                    for range_sub in nested_sub:
                        beg_line, end_line = range_sub
                        lines[beg_line] = '<ul>\n' + lines[beg_line]
                        if '\n</ol>' in lines[end_line]:
                            # insert the '\n</ul>' before '\n</ol>
                            lines[end_line] = lines[end_line].replace('\n</ol>','') + '\n</ul>\n</ol>'
                        else:
                            lines[end_line] = lines[end_line] + '\n</ul>'


            elif (i + 2 in unordered_list_lines and unordered_list_markers[i + 2] in list_markers):
                # find the last sequential simbol starting from i + 1 in unordered_list_markers that is in list_markers
                # and find the index of the last sequential simbol starting from i + 1 in unordered_list_markers that is in list_markers
                """ last_sequential_symbol = None
                last_sequential_index = None
                for j in range(i + 1, len(unordered_list_markers)):
                    if unordered_list_markers[j] in list_markers:
                        last_sequential_symbol = unordered_list_markers[j]
                        last_sequential_index = j
                    elif unordered_list_markers[j] not in list_markers:
                        break """
                my_marker = lines[i+2].strip()[0]
                range_start = i + 2

                _, _, last_sequential_symbol , last_sequential_index1, found_marker1, nested_list_markers1  = get_last_unordered_marker_in_sequence(text, my_marker, range_start, range_end)

                # # insert <li> and </li> before and after line[i]
                # #  insert <li> and </li> before and after line[i]
                # lines[i] = '<li>' + lines[i] + '</li>'

                # remove the </ol> in line[i + 1]
                lines[i + 1] = lines[i + 1].replace('</ol>', '')
                
                # for y in range(i+2, last_sequential_index1 + 1):
                #     lines[y] = '<li>' + lines[y] + '</li>'
                
                #  insert the <ul> before line[i+2]
                lines[i + 2] = '<ul>\n' + lines[i + 2]

                #  insert the </ul> after line[j + i + 1]
                # lines.insert(j + i + 1, '</ul>')
                lines[last_sequential_index1] = lines[last_sequential_index1] + '\n</ul>' + '\n</ol>'


                if found_marker1:
                    # check if all markers in nested_list_markers1 are the same
                    nested_sub = find_subranges(nested_list_markers1, unordered_list_markers, text, markerlist=['•', '‒', '▷'], level=1)

                    for range_sub in nested_sub:
                        beg_line, end_line = range_sub
                        lines[beg_line] = '<ul>\n' + lines[beg_line]
                        if '\n</ol>' in lines[end_line]:
                            # insert the '\n</ul>' before '\n</ol>
                            lines[end_line] = lines[end_line].replace('\n</ol>','') + '\n</ul>\n</ol>'
                        else:
                            lines[end_line] = lines[end_line] + '\n</ul>'

            else:
                # #  insert <li> and </li> before and after line[i]
                # lines[i] = '<li>' + lines[i] + '</li>'
                pass

    text = '\n'.join(lines)
    text = text.replace('\n\n', '\n')
    lines = text.split('\n')
    # special case !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    for i, line in enumerate(lines):
        stripped_line = line.strip()
        if any(is_unordered_list_item(stripped_line, level) for level in range(3)):
            if stripped_line[-1] == ':' and is_ordered_list_item(lines[i-2]) and lines[i-1] == '<ul>' and lines[i+1] == '</ul>' and lines[i+2] == '</ol>':
                lines[i-1] = ''
                lines[2] = '<ul>\n' + lines[2]
                lines[i+2] = ''
                lines[i+1] = ''
                lines[i-2] = lines[i-2] + '\n</ol>'
                lines[-1] = lines[-1] + '\n</ul>'
            pass
    # special case !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    text = '\n'.join(lines)

    while '\n\n' in text:
        text = text.replace('\n\n', '\n')


    # find if there are unordered list markers in the find_html_encapsulated_lines(text, ['ol', 'ul'])[1]
    while True:

        lines = text.split('\n')
        for v, i in enumerate(find_html_encapsulated_lines(text, ['ol', 'ul'])[0]):            
            stripped_line = lines[i].strip()
            if any(is_unordered_list_item(stripped_line, level) for level in range(3)):
                    
                my_marker = stripped_line[0]
                range_start = i
                range_end = len(lines)
                unordered_list_markers, unordered_list_lines, last_sequential_symbol , last_sequential_index, found_marker, nested_list_markers  = get_last_unordered_marker_in_sequence(text, my_marker, range_start, range_end)

                
                if found_marker:
                    # check if all markers in nested_list_markers are the same
                    nested_sub = find_subranges(nested_list_markers, unordered_list_markers, text, markerlist=['•', '‒', '▷'], level=1)
                    
                    for j,  range_sub in enumerate(nested_sub):                        
                        beg_line, end_line = range_sub

                        lines[beg_line] = '<ul>\n' + lines[beg_line]
                        if '\n</ol>' in lines[end_line]:
                            # insert the '\n</ul>' before '\n</ol>
                            lines[end_line] = lines[end_line].replace('\n</ol>','') + '\n</ul>\n</ol>'
                        else:
                            lines[end_line] = lines[end_line] + '\n</ul>'                        
                        if j == len(nested_sub) - 1: 
                            # if it is the last subrange:
                            # Append '<ul>\n' to the begining of lines[i]
                            lines[i] = '<ul>\n' + lines[i]                            
                            # append the </ul> of the first line to the end of the last line^
                            # if '\n</ol>' in lines[end_line]:
                            #     # insert the '\n</ul>' before '\n</ol>
                            #     lines[end_line] = lines[end_line].replace('\n</ol>','') + '\n</ul>\n</ol>'
                            # else:
                            #     lines[end_line] = lines[end_line] + '\n</ul>'

                            if '\n</ol>' in lines[last_sequential_index]:
                                lines[last_sequential_index] = lines[last_sequential_index].replace('\n</ol>','') + '\n</ul>\n</ol>'
                            else:
                                lines[last_sequential_index] = lines[last_sequential_index] + '\n</ul>'
                elif last_sequential_index and not found_marker:
                    # Append '<ul>\n' to the begining of lines[i]
                    lines[i] = '<ul>\n' + lines[i]
                    if '\n</ol>' in lines[last_sequential_index]:
                        lines[last_sequential_index] = lines[last_sequential_index].replace('\n</ol>','') + '\n</ul>\n</ol>'
                    else:
                        lines[last_sequential_index] = lines[last_sequential_index] + '\n</ul>'

                # else:
                #     # Append '<ul>\n' to the begining of lines[i]  and '</ul>' to the end of lines[i]
                #     lines[i] = '<ul>\n' + lines[i] + '\n</ul>'
                
                

                break
        text = '\n'.join(lines)
        while '\n\n' in text:
            text = text.replace('\n\n', '\n')

        if not any(is_unordered_list_item(line, level) for level in range(3) for line in find_html_encapsulated_lines(text, ['ol', 'ul'])[1]):
            break


    text = '\n'.join(lines)
    while '\n\n' in text:
        text = text.replace('\n\n', '\n')
    lines = text.split('\n')
    html_lines = []
    html_block = ''
    for line in lines:
        stripped_line = line.strip()
        if stripped_line == '' or stripped_line == ' ':
            pass
        elif any(is_unordered_list_item(stripped_line, level) for level in range(3)):
            if '<Start_nested_text>' in stripped_line:
                #  find the index of the first '<Start_nested_text>' in stripped_line
                start_index = stripped_line.find('<Start_nested_text></b>')
                if '</b></i>' in stripped_line:
                    html_lines.append('<li>' + re.sub(r'^\s*[•‒▷]\s*', '',stripped_line[:start_index + len('<Start_nested_text></b></i>')].replace('<Start_nested_text></b></i>','</b></i>'), flags=re.IGNORECASE) + '</li>')

                    nested_text = stripped_line[start_index + len('<Start_nested_text></b></i>'):]
                    if(len(nested_text) > 0):
                        html_lines.append('<p>' + nested_text + '</p>')
                elif '</b>' in stripped_line:
                    html_lines.append('<li>' + re.sub(r'^\s*[•‒▷]\s*', '',stripped_line[:start_index + len('<Start_nested_text></b>')].replace('<Start_nested_text></b>','</b>'), flags=re.IGNORECASE) + '</li>')

                    nested_text = stripped_line[start_index + len('<Start_nested_text></b>'):]
                    if(len(nested_text) > 0):
                        html_lines.append('<p>' + nested_text + '</p>')
                elif '</i>' in stripped_line:
                    html_lines.append('<li>' + re.sub(r'^\s*[•‒▷]\s*', '',stripped_line[:start_index + len('<Start_nested_text></i>')].replace('<Start_nested_text></i>','</i>'), flags=re.IGNORECASE) + '</li>')

                    nested_text = stripped_line[start_index + len('<Start_nested_text></i>'):]
                    if(len(nested_text) > 0):
                        html_lines.append('<p>' + nested_text + '</p>')

            else:
                html_lines.append('<li>' + re.sub(r'^\s*[•‒▷]\s*', '', stripped_line) + '</li>')
        
        elif is_ordered_list_item(stripped_line):
            html_lines.append('<li>' + re.sub(r'^\s*\d+[.)]\s*|^(\s*(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})[.)]\s*)', '', stripped_line, flags=re.IGNORECASE) + '</li>')

        elif 'ul>' not in stripped_line and 'ol>' not in stripped_line and 'i>' not in stripped_line and 'p>' not in stripped_line:
            html_lines.append('<p>' + stripped_line + '</p>')
        else:
            html_lines.append(stripped_line)
    html_block = '\n'.join(html_lines)    
    return html_block
                                
def get_dataDict(pageNr, pdfFile, area):
    doc = fitz.open(pdfFile)
    page = doc[pageNr]
    blockDict = page.get_text('dict')['blocks']

    # Filter the blocks based on the area
    filtered_blocks = [block for block in blockDict if is_in_area(block['bbox'], area)]

    # close the pdf file
    doc.close()

    return filtered_blocks

def is_in_area(bbox, area):
    x, y, x2, y2 = bbox
    ax, ay, ax2, ay2 = area

    # Check if the block is within the area
    return ax <= x < ax2 and ay <= y < ay2

def sort_key(block):
    # Get the coordinates of the block
    x, y, x2, y2 = block['bbox']

    # Check which area the block is in and assign a sort key accordingly
    if x < 690:
        area = 0
    else:
        area = 1

    # Return a tuple that first sorts by area, then by y-coordinate (top to bottom), and then by x-coordinate (left to right)
    return (area, y, x)

def find_last_list_bold_span(block):
    for k, line in enumerate(block['lines']):
        if (any(is_unordered_list_item(line['spans'][0]['text'].strip(), level) for level in range(3)) or is_ordered_list_item(line['spans'][0]['text'].strip())) and rect_in_rect(fitz.Rect(*line['spans'][0]['bbox']), fitz.Rect(*block['bbox'])):
            in_bold_sequence = False
            last_bold_span = None
            
            for t, span in enumerate(line['spans']):
                if k != 0 and t != 0:
                    if 'Bold' in span['font']:
                        in_bold_sequence = True
                        last_bold_span = (k, t)
                    else:
                        in_bold_sequence = False

                # If this is the last span of the line and it's bold and its right edge is within the block
                if t == len(line['spans']) - 1 and in_bold_sequence and span['bbox'][2] < block['bbox'][2]:
                    return last_bold_span

    return (None, None)  # Return None if no such span is found

""" # get the html data for each position in each page of the pdf
def get_data_html(pgList='all', pdfFile=pdfFile):
    # This function takes:
    # - a list of pages (pgList) or 'all' to iterate through all the pages
    # - a pdf file (pdfFile)
   
    data_places = draw_the_comb_rect(pgList, pdfFile)
    
    resultDict = {}

    

    doc = fitz.open(pdfFile)

    for i, value in data_places.items():
        print(i)
        page = doc[i]
        product_description = ''
        company_description = ''

        blockDict = page.get_text('dict')['blocks']

        # reorder the 'blockDict' so that blocks within data_places[i]['comp_desc'] are ordered from top to bottom and left to write, divided into two areas:
        # data_places[i]['comp_desc'][0], data_places[i]['comp_desc'][1], 690, data_places[i]['comp_desc'][3]
        # and 690, data_places[i]['comp_desc'][1], data_places[i]['comp_desc'][2], data_places[i]['comp_desc'][3]
        # and within these areas, if there are side-by-side blocks, they are ordered from top to bottom and from left to right

        # Sort the blocks
        blockDict = sorted(blockDict, key=sort_key)
        

        for block in blockDict:

            for items in value.items():
                key, bbox = items

                if key == 'comp_name':
                    if block['type'] == 0 and bbox.contains(fitz.Rect(*block['bbox'])):
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans']]
                        company_name = '<h2>' + ' '.join(text).strip() + '</h2>' 
                        # data_places[i]['comp_name'] = company_name
                        # print('------------------------------------------------ \n company_name \n', company_name)
                elif key == 'comp_url':
                    if block['type'] == 0 and bbox.contains(fitz.Rect(*block['bbox'])):
                        text = [span['text'] for line in block['lines'] for span in line['spans']]
                        company_url = f"<a href=\"https://{' '.join(text).strip()}\" target=\"_blank\">{' '.join(text).strip()}</a>"
                        # data_places[i]['comp_url'] = company_url
                        # print('\n company_url \n', company_url)
                elif key == 'sol_name':
                    if block['type'] == 0 and bbox.intersects(fitz.Rect(*block['bbox'])):
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans'] if span['color'] == 15827]
                        solution_name = '<h1>' + ' '.join(text).strip() + '</h1>'
                        # data_places[i]['sol_name'] = solution_name
                        # print('\n solution_name \n', solution_name)
                elif key == 'scope':
                    scope =[]
                    if block['type'] == 0 and bbox.contains(fitz.Rect(*block['bbox'])):
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans']]
                        scope.append(' '.join(text).strip())
                        # data_places[i]['scope'] = scope
                        # print('\n scope \n', scope)
                elif key == 'destType':
                     destType = get_infografic_text_data(page, bbox, area_threshold=1000)
                elif key == 'solType':
                    solType = get_infografic_text_data(page, bbox, area_threshold=1000)
                elif key == 'comp_desc':
                    if block['type'] == 0 and bbox.contains(fitz.Rect(*block['bbox'])):                    
                        text = []
                        previous_line = ''
                        line_text = ''

                        target_line, target_span = find_last_list_bold_span(block)
                        for k, line in enumerate(block['lines']):
                            for m, span in enumerate(line['spans']):
                                if rect_in_rect(fitz.Rect(*span['bbox']), bbox):
                                    # if the span is the last in the line, and the span bbox is < than the block bbox, add a new line at th end of the span
                                    # and the span ends with a '.', ':', '!', '?' add a new line at the end of the span
                                    # else it is in the same paragraph
                                    if k == target_line and m == target_span:
                                        span_text = span['text'] + '<Start_nested_text>'
                                    elif k == len(block['lines']) -1 and m == len(line['spans']) - 1 and span['bbox'][2] < bbox[2] and span['text'] != ' ' and span['text'].strip()[-1] in ['.', ':', '!', '?']:                                        
                                        span_text = span['text'] + '\n'
                                    else:
                                        span_text = span['text']#.strip()
                                    
                                    if len(block['lines']) == 1 and len(line['spans']) == 1 and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                        # if the block only has 1 line with one bold span is is a title, so add a new line at the end of the span
                                        span_text = span['text'] + '\n'   

                                    # if k == 1 and len(block['lines']) == 2 and len(line['spans']) == 1 and len(block['lines'][k-1]['spans']) == 1 and any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)) and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                    #     span_text = span['text'] + '<Start_nested_text>'

                                    # exceptions of titles, mixed regular and bold, more than one line,
                                   
                                    #  i found an additional marker for bullets '•' in the text, so i need to normalize it
                                    span_text = span_text.replace(' ا', '•')
                                    span_text = span_text.replace('ا', '•')
                                        
                                    for j in range(0,2):
                                        if  m == 0 and (is_ordered_list_item(span_text) or is_unordered_list_item(span_text, j)):
                                            # Replace Roman numeral list markers with Arabic numeral list markers
                                            span_text = re.sub(r'^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s', lambda match: str(roman_to_int(match.group(0).rstrip('. '))) + '. ', span_text)
                                            # Find the first character that is not a bullet
                                            match = re.search(r'(\d+\.\s|\d+\)\s|(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s|[•‒▷]\s)', span_text)
                                            if match:
                                                k = match.end()
                                            else:
                                                k = len(span_text)#0

                                            # normalize ol to nr+'.' if nr+')'
                                            if ')' in span_text[:k]:
                                                span_text = span_text[:k].replace(')', '.') + span_text[k:]

                                            # Apply HTML tags to the rest of the text
                                            if 'BoldItalic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b><i>{span_text[k:]}</i></b>"
                                            elif 'Bold' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b>{span_text[k:]}</b>"
                                            elif 'Italic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<i>{span_text[k:]}</i>"
                                            else:
                                                line_text += '\n' + span_text
                                                # if m == 0:
                                                #     line_text += '\n' + span_text
                                                # else:
                                                #     line_text += span_text
                                        else:
                                            if 'BoldItalic' in span['font']:
                                                line_text += f"<b><i>{span_text}</i></b>"
                                            elif 'Bold' in span['font']:
                                                line_text += f"<b>{span_text}</b>"
                                            elif 'Italic' in span['font']:
                                                line_text += f"<i>{span_text}</i>"
                                            else:
                                                line_text += span_text
                                        break


                            if is_ordered_list_item(line_text) or is_unordered_list_item(line_text, 0):
                                if previous_line:
                                    text.append(previous_line)
                                previous_line = line_text
                            elif is_unordered_list_item(line_text, 1) or is_unordered_list_item(line_text, 2):
                                previous_line += '\n' + line_text
                            else:
                                previous_line += ' ' + line_text
                            line_text = ''
                        # Append the last line if it's not empty
                        if previous_line:
                            text.append(previous_line)

                        # Add a new line after the first period in the end of a line, encountered after the last bullet point
                        # text = [line.replace('.', '.\n', 1) if (is_ordered_list_item(line) or is_unordered_list_item(line, 0) or is_unordered_list_item(line, 1) or is_unordered_list_item(line, 2)) else line for line in text]#'•' not in line else line for line in text]

                        text_block = '\n'.join(text)

                        company_description += text_block
                elif key == 'sol_desc':
                    if block['type'] == 0 and bbox.contains(fitz.Rect(*block['bbox'])):                    
                        text = []
                        previous_line = ''
                        line_text = ''

                        target_line, target_span = find_last_list_bold_span(block)
                        for k, line in enumerate(block['lines']):
                            for m, span in enumerate(line['spans']):
                                # if 'INFO POINT' in solution_name:
                                #     if span['text'] == 'Digital touchscreen':
                                #         print(block)
                                #         print('span[bbox]', span['bbox'])
                                #         print('block[bbox]',block['bbox'])
                                #         print('bbox', bbox)
                                #         print('here we go 0!', block['lines'][k-1]['spans'][0]['text'].strip(), any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)))
                                #         if span['bbox'][2] < (0.8 * bbox[2]):
                                #             print('here we go 1!', block['lines'][k-1]['spans'][0]['text'].strip(), any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)))

                                    # print(m, 'span_text', span_text, block['number'])
                                    # print(m, 'here we go!', ''.join(text.strip() for span in line['spans'] for text in span['text']))
                                if rect_in_rect(fitz.Rect(*span['bbox']), bbox):
                                    # if the span is the last in the line, and the span bbox is < than the block bbox, add a new line at th end of the span
                                    # and the span ends with a '.', ':', '!', '?' add a new line at the end of the span
                                    # else it is in the same paragraph
                                    if k == target_line and m == target_span:
                                        span_text = span['text'] + '<Start_nested_text>'
                                    elif k == len(block['lines']) -1 and m == len(line['spans']) - 1 and span['bbox'][2] < bbox[2] and span['text'] != ' ' and span['text'].strip()[-1] in ['.', ':', '!', '?']:                                        
                                        span_text = span['text'] + '\n'
                                    else:
                                        span_text = span['text']#.strip()
                                    
                                    if len(block['lines']) == 1 and len(line['spans']) == 1 and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                        # if the block only has 1 line with one bold span is is a title, so add a new line at the end of the span
                                        span_text = span['text'] + '\n'   

                                    # if k == 1 and len(block['lines']) == 2 and len(line['spans']) == 1 and len(block['lines'][k-1]['spans']) == 1 and any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)) and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                    #     span_text = span['text'] + '<Start_nested_text>'

                                    # exceptions of titles, mixed regular and bold, more than one line,
                                    if span_text == 'More than 150 KPIs available' or span_text == 'Differential value of artificial intelligence in the digital ' or span_text == 'One of the fundamental reasons to digitalise is the opportunities ':                                        
                                        span_text = '\n' + span_text

                                    #  i found an additional marker for bullets '•' in the text, so i need to normalize it
                                    span_text = span_text.replace(' ا', '•')
                                    span_text = span_text.replace('ا', '•')
                                        
                                    for j in range(0,2):
                                        if  m == 0 and (is_ordered_list_item(span_text) or is_unordered_list_item(span_text, j)):
                                            # Replace Roman numeral list markers with Arabic numeral list markers
                                            span_text = re.sub(r'^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s', lambda match: str(roman_to_int(match.group(0).rstrip('. '))) + '. ', span_text)
                                            # Find the first character that is not a bullet
                                            match = re.search(r'(\d+\.\s|\d+\)\s|(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s|[•‒▷]\s)', span_text)
                                            if match:
                                                k = match.end()
                                            else:
                                                k = len(span_text)#0

                                            # normalize ol to nr+'.' if nr+')'
                                            if ')' in span_text[:k]:
                                                span_text = span_text[:k].replace(')', '.') + span_text[k:]

                                            # Apply HTML tags to the rest of the text
                                            if 'BoldItalic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b><i>{span_text[k:]}</i></b>"
                                            elif 'Bold' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b>{span_text[k:]}</b>"
                                            elif 'Italic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<i>{span_text[k:]}</i>"
                                            else:
                                                line_text += '\n' + span_text
                                                # if m == 0:
                                                #     line_text += '\n' + span_text
                                                # else:
                                                #     line_text += span_text
                                        else:
                                            if 'BoldItalic' in span['font']:
                                                line_text += f"<b><i>{span_text}</i></b>"
                                            elif 'Bold' in span['font']:
                                                line_text += f"<b>{span_text}</b>"
                                            elif 'Italic' in span['font']:
                                                line_text += f"<i>{span_text}</i>"
                                            else:
                                                line_text += span_text
                                        break


                            if is_ordered_list_item(line_text) or is_unordered_list_item(line_text, 0):
                                if previous_line:
                                    text.append(previous_line)
                                previous_line = line_text
                            elif is_unordered_list_item(line_text, 1) or is_unordered_list_item(line_text, 2):
                                previous_line += '\n' + line_text
                            else:
                                previous_line += ' ' + line_text
                            line_text = ''
                        # Append the last line if it's not empty
                        if previous_line:
                            text.append(previous_line)

                        # Add a new line after the first period in the end of a line, encountered after the last bullet point
                        # text = [line.replace('.', '.\n', 1) if (is_ordered_list_item(line) or is_unordered_list_item(line, 0) or is_unordered_list_item(line, 1) or is_unordered_list_item(line, 2)) else line for line in text]#'•' not in line else line for line in text]

                        text_block = '\n'.join(text)

                        product_description += text_block
        
        # get the logo for the page
        # print(data_places[i])
        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap(clip=data_places[i]['logo'])
        data = pix.samples
        # create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)

        # Save the PIL image to a BytesIO stream in PNG format
        img_stream = io.BytesIO()
        img.save(img_stream, format='PNG')

        # Get the image file data as a byte string
        img_data = img_stream.getvalue()

        # convert the image bytes to base64
        company_img_base64 = base64.b64encode(img_data).decode('utf-8')

        #==========================================================
        # Create a data URL for the imge
        data_url = f"data:image/png;base64,{company_img_base64}"

        # Now you can use `data_url` in an img tag
        company_logo = f'<br><img src="{data_url}" alt="Logo">'

        # print('company_logo', company_logo)                        
                            



    	
        #  cLEANENG THE TEXT FOR HTML encapsulating
        
        # product_description = remove_redundant_tags(product_description)
        while '  ' in product_description:
            product_description = product_description.replace('  ', ' ')
        while ' \n' in product_description:
            product_description = product_description.replace(' \n', '\n')
        while '\n ' in product_description:
            product_description = product_description.replace('\n ', '\n')
            
        while '<b>\n' in product_description:
            product_description = product_description.replace('<b>\n', '\n<b>')
        while '<b> </b>' in product_description:
            product_description = product_description.replace('<b> </b>', '')
        while '</b><b>' in product_description:
            product_description = product_description.replace('</b><b>', '')
        while '</b> <b>' in product_description:
            product_description = product_description.replace('</b> <b>', '')
        while '</i><i>' in product_description:
            product_description = product_description.replace('</i><i>', '')
        while '</i> <i>' in product_description:
            product_description = product_description.replace('</i> <i>', '')            

        while '\n</b>' in product_description:
            product_description = product_description.replace('\n</b>', '</b>\n')
        while '\n</i>' in product_description:
            product_description = product_description.replace('\n</i>', '</i>\n')
        while '\n</i></b>' in product_description:
            product_description = product_description.replace('\n</i></b>', '</i></b>\n')
        while '\n\n' in product_description:
            product_description = product_description.replace('\n\n', '\n')
        while '\n \n' in product_description:
            product_description = product_description.replace('\n \n', '\n')
        # print('\n product_description_V4 \n', text_to_html6(handle_ordered_lists(product_description)))

        # company_description = remove_redundant_tags(company_description)
        while '  ' in company_description:
            company_description = company_description.replace('  ', ' ')
        while ' \n' in company_description:
            company_description = company_description.replace(' \n', '\n')
        while '\n ' in company_description:
            company_description = company_description.replace('\n ', '\n')
            
        while '<b>\n' in company_description:
            company_description = company_description.replace('<b>\n', '\n<b>')
        while '<b> </b>' in company_description:
            company_description = company_description.replace('<b> </b>', '')
        while '</b><b>' in company_description:
            company_description = company_description.replace('</b><b>', '')
        while '</b> <b>' in company_description:
            company_description = company_description.replace('</b> <b>', '')
        while '</i><i>' in company_description:
            company_description = company_description.replace('</i><i>', '')
        while '</i> <i>' in company_description:
            company_description = company_description.replace('</i> <i>', '')            

        while '\n</b>' in company_description:
            company_description = company_description.replace('\n</b>', '</b>\n')
        while '\n</i>' in company_description:
            company_description = company_description.replace('\n</i>', '</i>\n')
        while '\n</i></b>' in company_description:
            company_description = company_description.replace('\n</i></b>', '</i></b>\n')
        while '\n\n' in company_description:
            company_description = company_description.replace('\n\n', '\n')
        while '\n \n' in company_description:
            company_description = company_description.replace('\n \n', '\n')
        # print('\n product_description_V4 \n', text_to_html6(handle_ordered_lists(company_description)))


        # ------------------------------------------------------------------------------------------------------------------------------------------------------------------

        # global resultDict
        if 'companies' not in resultDict:
            resultDict['companies'] = {}
        # Add the company and product to the dictionary
        if company_name in resultDict['companies']:
            # If the dictionary exists, append 'element' to the list
            resultDict['companies'][company_name]['products'].append({
                'page':i+1, 
                'product_name': solution_name, 
                'product_description': text_to_html6(handle_ordered_lists(product_description)), 
                'target_audience':'',
                'scope': scope, 
                'destType':destType,
                'solType':solType})

            resultDict['companies'][company_name]['company_description'] = text_to_html6(handle_ordered_lists(company_description))
            resultDict['companies'][company_name]['company_url'] = company_url
            resultDict['companies'][company_name]['phone'] = ''
            resultDict['companies'][company_name]['email'] = ''
            # resultDict['companies'][company_name]['address'] = ''
            # resultDict['companies'][company_name]['geoRef'] = ''


            resultDict['companies'][company_name]['company_logo'] = company_logo
            resultDict['companies'][company_name]['company_QR'] = ''

          
        else:
            # If the dictionary doesn't exist, create a new one
            resultDict['companies'][company_name] = {
                'company_description': text_to_html6(handle_ordered_lists(company_description)),
                'company_url': company_url,
                'phone': '',
                'email': '',
                'address':'',
                'geoRef':'',                
                'company_logo': company_logo,
                'company_QR': '',
                
                'products': [
                    {
                        'page': i+1,
                        'product_name': solution_name,
                        'product_description': text_to_html6(handle_ordered_lists(product_description)),
                        'target_audience':'',
                        'scope': scope,
                        'destType': destType,
                        'solType': solType
                    }
                ]
}

    return resultDict

result = get_data_html(pgList=range(9,363), pdfFile=pdfFile)
# result = get_data_html(pgList=range(9,15), pdfFile=pdfFile)

# Save the results to a JSON file
with open('1stSpanish/3rdED/Catalogue Spanish_3.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4) """

In [ ]:
# get the html data for each position in each page of the pdf
def get_data_html(pgList='all', pdfFile=pdfFile):
    """ This function takes:
    - a list of pages (pgList) or 'all' to iterate through all the pages
    - a pdf file (pdfFile)
    """
    data_places = draw_the_comb_rect(pgList, pdfFile)
    
    resultDict = {}

    

    doc = fitz.open(pdfFile)

    for i, value in data_places.items():
        # print(i)
        page = doc[i]
        product_description = ''
        company_description = ''


        for items in value.items():
            key, bbox = items

            if key == 'comp_name':

                blockDict = get_dataDict(i, pdfFile, bbox)                
                for block in blockDict:
                    if block['type'] == 0:
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans']]
                        company_name = '<h2>' + ' '.join(text).strip() + '</h2>' 
                # print('------------------------------------------------ \n company_name \n', company_name)
            elif key == 'comp_url':

                blockDict = get_dataDict(i, pdfFile, bbox)                
                for block in blockDict:
                    if block['type'] == 0:
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans']]
                        company_url = f"<a href=\"https://{' '.join(text).strip()}\" target=\"_blank\">{' '.join(text).strip()}</a>"
                # print('company_url', company_url)
            elif key == 'sol_name':

                blockDict = get_dataDict(i, pdfFile, bbox)                
                for block in blockDict:
                    if block['type'] == 0:
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans'] if span['color'] == 15827]
                        solution_name = '<h1>' + ' '.join(text).strip() + '</h1>'
                # print('solution_name', solution_name)
            elif key == 'scope':
                blockDict = get_dataDict(i, pdfFile, bbox)
                scope_list =[]                
                for block in blockDict:
                    if block['type'] == 0:
                        text = [span['text'].strip() for line in block['lines'] for span in line['spans'] if span['color'] != 7171696]
                        scope_list.append(' '.join(text).strip())
                    
                scope_list = [item for item in scope_list if item != '']
                # Append 'Scope:' to the first element of the list
                scope_list[0] = 'Scope: ' + "\n• " + scope_list[0]

                text_block_data = "\n• ".join(scope_list)

                scope = text_to_html6(text_block_data)




                # print('scope', scope)
            elif key == 'destType':
                destType_list = get_infografic_list_data(page, bbox, area_threshold=1000)

                # Append 'Destination type:' to the first element of the list
                destType_list[0] = 'Destination type: ' + "\n• " + destType_list[0]

                text_block_data = "\n• ".join(destType_list)

                destType = text_to_html6(text_block_data)

                print('destType', destType)

            elif key == 'solType':
                solType_list = get_infografic_list_data(page, bbox, area_threshold=1000)

                # Append 'Solution type:' to the first element of the list
                solType_list[0] = 'Solution type: ' + "\n• " + solType_list[0]

                text_block_data = "\n• ".join(solType_list)

                solType = text_to_html6(text_block_data)

                # print('solType', solType)
            elif key == 'comp_desc':

                blockDict = get_dataDict(i, pdfFile, bbox)                
                for block in blockDict:
                    if block['type'] == 0:                 
                        text = []
                        previous_line = ''
                        line_text = ''

                        target_line, target_span = find_last_list_bold_span(block)
                        for k, line in enumerate(block['lines']):
                            for m, span in enumerate(line['spans']):
                                if rect_in_rect(fitz.Rect(*span['bbox']), bbox):
                                    # if the span is the last in the line, and the span bbox is < than the block bbox, add a new line at th end of the span
                                    # and the span ends with a '.', ':', '!', '?' add a new line at the end of the span
                                    # else it is in the same paragraph
                                    if k == target_line and m == target_span:
                                        span_text = span['text'] + '<Start_nested_text>'
                                    elif k == len(block['lines']) -1 and m == len(line['spans']) - 1 and span['bbox'][2] < bbox[2] and span['text'] != ' ' and span['text'].strip()[-1] in ['.', ':', '!', '?']:                                        
                                        span_text = span['text'] + '\n'
                                    else:
                                        span_text = span['text']#.strip()
                                    
                                    if len(block['lines']) == 1 and len(line['spans']) == 1 and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                        # if the block only has 1 line with one bold span is is a title, so add a new line at the end of the span
                                        span_text = span['text'] + '\n'   

                                    # if k == 1 and len(block['lines']) == 2 and len(line['spans']) == 1 and len(block['lines'][k-1]['spans']) == 1 and any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)) and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                    #     span_text = span['text'] + '<Start_nested_text>'

                                    # exceptions of titles, mixed regular and bold, more than one line,
                                    
                                    #  i found an additional marker for bullets '•' in the text, so i need to normalize it
                                    span_text = span_text.replace(' ا', '•')
                                    span_text = span_text.replace('ا', '•')
                                        
                                    for j in range(0,2):
                                        if  m == 0 and (is_ordered_list_item(span_text) or is_unordered_list_item(span_text, j)):
                                            # Replace Roman numeral list markers with Arabic numeral list markers
                                            span_text = re.sub(r'^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s', lambda match: str(roman_to_int(match.group(0).rstrip('. '))) + '. ', span_text)
                                            # Find the first character that is not a bullet
                                            match = re.search(r'(\d+\.\s|\d+\)\s|(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s|[•‒▷]\s)', span_text)
                                            if match:
                                                k = match.end()
                                            else:
                                                k = len(span_text)#0

                                            # normalize ol to nr+'.' if nr+')'
                                            if ')' in span_text[:k]:
                                                span_text = span_text[:k].replace(')', '.') + span_text[k:]

                                            # Apply HTML tags to the rest of the text
                                            if 'BoldItalic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b><i>{span_text[k:]}</i></b>"
                                            elif 'Bold' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b>{span_text[k:]}</b>"
                                            elif 'Italic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<i>{span_text[k:]}</i>"
                                            else:
                                                line_text += '\n' + span_text
                                                # if m == 0:
                                                #     line_text += '\n' + span_text
                                                # else:
                                                #     line_text += span_text
                                        else:
                                            if 'BoldItalic' in span['font']:
                                                line_text += f"<b><i>{span_text}</i></b>"
                                            elif 'Bold' in span['font']:
                                                line_text += f"<b>{span_text}</b>"
                                            elif 'Italic' in span['font']:
                                                line_text += f"<i>{span_text}</i>"
                                            else:
                                                line_text += span_text
                                        break


                            if is_ordered_list_item(line_text) or is_unordered_list_item(line_text, 0):
                                if previous_line:
                                    text.append(previous_line)
                                previous_line = line_text
                            elif is_unordered_list_item(line_text, 1) or is_unordered_list_item(line_text, 2):
                                previous_line += '\n' + line_text
                            else:
                                previous_line += ' ' + line_text
                            line_text = ''
                        # Append the last line if it's not empty
                        if previous_line:
                            text.append(previous_line)

                        # Add a new line after the first period in the end of a line, encountered after the last bullet point
                        # text = [line.replace('.', '.\n', 1) if (is_ordered_list_item(line) or is_unordered_list_item(line, 0) or is_unordered_list_item(line, 1) or is_unordered_list_item(line, 2)) else line for line in text]#'•' not in line else line for line in text]

                        text_block = '\n'.join(text)

                        company_description += text_block
                # print('company_description', company_description)        
            elif key == 'sol_desc':

                blockDict = get_dataDict(i, pdfFile, bbox)                
                for block in blockDict:
                    if block['type'] == 0:                   
                        text = []
                        previous_line = ''
                        line_text = ''

                        target_line, target_span = find_last_list_bold_span(block)
                        for k, line in enumerate(block['lines']):
                            for m, span in enumerate(line['spans']):
                                # if 'INFO POINT' in solution_name:
                                #     if span['text'] == 'Digital touchscreen':
                                #         print(block)
                                #         print('span[bbox]', span['bbox'])
                                #         print('block[bbox]',block['bbox'])
                                #         print('bbox', bbox)
                                #         print('here we go 0!', block['lines'][k-1]['spans'][0]['text'].strip(), any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)))
                                #         if span['bbox'][2] < (0.8 * bbox[2]):
                                #             print('here we go 1!', block['lines'][k-1]['spans'][0]['text'].strip(), any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)))

                                    # print(m, 'span_text', span_text, block['number'])
                                    # print(m, 'here we go!', ''.join(text.strip() for span in line['spans'] for text in span['text']))
                                if rect_in_rect(fitz.Rect(*span['bbox']), bbox):
                                    # if the span is the last in the line, and the span bbox is < than the block bbox, add a new line at th end of the span
                                    # and the span ends with a '.', ':', '!', '?' add a new line at the end of the span
                                    # else it is in the same paragraph
                                    if k == target_line and m == target_span:
                                        span_text = span['text'] + '<Start_nested_text>'
                                    elif k == len(block['lines']) -1 and m == len(line['spans']) - 1 and span['bbox'][2] < bbox[2] and span['text'] != ' ' and span['text'].strip()[-1] in ['.', ':', '!', '?']:                                        
                                        span_text = span['text'] + '\n'
                                    else:
                                        span_text = span['text']#.strip()
                                    
                                    if len(block['lines']) == 1 and len(line['spans']) == 1 and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                        # if the block only has 1 line with one bold span is is a title, so add a new line at the end of the span
                                        span_text = span['text'] + '\n'   

                                    # if k == 1 and len(block['lines']) == 2 and len(line['spans']) == 1 and len(block['lines'][k-1]['spans']) == 1 and any(is_unordered_list_item(block['lines'][k-1]['spans'][0]['text'].strip(), level) for level in range(3)) and span['bbox'][2] < bbox[2] and 'Bold' in span['font']:
                                    #     span_text = span['text'] + '<Start_nested_text>'

                                    # exceptions of titles, mixed regular and bold, more than one line,
                                    if span_text == 'More than 150 KPIs available' or span_text == 'Differential value of artificial intelligence in the digital ' or span_text == 'One of the fundamental reasons to digitalise is the opportunities ':                                        
                                        span_text = '\n' + span_text

                                    #  i found an additional marker for bullets '•' in the text, so i need to normalize it
                                    span_text = span_text.replace(' ا', '•')
                                    span_text = span_text.replace('ا', '•')
                                        
                                    for j in range(0,2):
                                        if  m == 0 and (is_ordered_list_item(span_text) or is_unordered_list_item(span_text, j)):
                                            # Replace Roman numeral list markers with Arabic numeral list markers
                                            span_text = re.sub(r'^(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s', lambda match: str(roman_to_int(match.group(0).rstrip('. '))) + '. ', span_text)
                                            # Find the first character that is not a bullet
                                            match = re.search(r'(\d+\.\s|\d+\)\s|(M{0,3})(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s|[•‒▷]\s)', span_text)
                                            if match:
                                                k = match.end()
                                            else:
                                                k = len(span_text)#0

                                            # normalize ol to nr+'.' if nr+')'
                                            if ')' in span_text[:k]:
                                                span_text = span_text[:k].replace(')', '.') + span_text[k:]

                                            # Apply HTML tags to the rest of the text
                                            if 'BoldItalic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b><i>{span_text[k:]}</i></b>"
                                            elif 'Bold' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<b>{span_text[k:]}</b>"
                                            elif 'Italic' in span['font'] and k != len(span_text):
                                                line_text += '\n' + span_text[:k] + f"<i>{span_text[k:]}</i>"
                                            else:
                                                line_text += '\n' + span_text
                                                # if m == 0:
                                                #     line_text += '\n' + span_text
                                                # else:
                                                #     line_text += span_text
                                        else:
                                            if 'BoldItalic' in span['font']:
                                                line_text += f"<b><i>{span_text}</i></b>"
                                            elif 'Bold' in span['font']:
                                                line_text += f"<b>{span_text}</b>"
                                            elif 'Italic' in span['font']:
                                                line_text += f"<i>{span_text}</i>"
                                            else:
                                                line_text += span_text
                                        break


                            if is_ordered_list_item(line_text) or is_unordered_list_item(line_text, 0):
                                if previous_line:
                                    text.append(previous_line)
                                previous_line = line_text
                            elif is_unordered_list_item(line_text, 1) or is_unordered_list_item(line_text, 2):
                                previous_line += '\n' + line_text
                            else:
                                previous_line += ' ' + line_text
                            line_text = ''
                        # Append the last line if it's not empty
                        if previous_line:
                            text.append(previous_line)

                        # Add a new line after the first period in the end of a line, encountered after the last bullet point
                        # text = [line.replace('.', '.\n', 1) if (is_ordered_list_item(line) or is_unordered_list_item(line, 0) or is_unordered_list_item(line, 1) or is_unordered_list_item(line, 2)) else line for line in text]#'•' not in line else line for line in text]

                        text_block = '\n'.join(text)

                        product_description += text_block
                print('product_description', product_description)
        
        # get the logo for the page
        # print(data_places[i])
        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap(clip=data_places[i]['logo'])
        data = pix.samples
        # create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)

        # Save the PIL image to a BytesIO stream in PNG format
        img_stream = io.BytesIO()
        img.save(img_stream, format='PNG')

        # Get the image file data as a byte string
        img_data = img_stream.getvalue()

        # convert the image bytes to base64
        company_img_base64 = base64.b64encode(img_data).decode('utf-8')

        #==========================================================
        # Create a data URL for the imge
        data_url = f"data:image/png;base64,{company_img_base64}"

        # Now you can use `data_url` in an img tag
        company_logo = f'<br><img src="{data_url}" alt="Logo">'

        # print('company_logo', company_logo)                        
                            



    	
        #  cLEANENG THE TEXT FOR HTML encapsulating
        
        # product_description = remove_redundant_tags(product_description)
        while '  ' in product_description:
            product_description = product_description.replace('  ', ' ')
        while ' \n' in product_description:
            product_description = product_description.replace(' \n', '\n')
        while '\n ' in product_description:
            product_description = product_description.replace('\n ', '\n')
            
        while '<b>\n' in product_description:
            product_description = product_description.replace('<b>\n', '\n<b>')
        while '<b> </b>' in product_description:
            product_description = product_description.replace('<b> </b>', '')
        while '</b><b>' in product_description:
            product_description = product_description.replace('</b><b>', '')
        while '</b> <b>' in product_description:
            product_description = product_description.replace('</b> <b>', '')
        while '</i><i>' in product_description:
            product_description = product_description.replace('</i><i>', '')
        while '</i> <i>' in product_description:
            product_description = product_description.replace('</i> <i>', '')            

        while '\n</b>' in product_description:
            product_description = product_description.replace('\n</b>', '</b>\n')
        while '\n</i>' in product_description:
            product_description = product_description.replace('\n</i>', '</i>\n')
        while '\n</i></b>' in product_description:
            product_description = product_description.replace('\n</i></b>', '</i></b>\n')
        while '\n\n' in product_description:
            product_description = product_description.replace('\n\n', '\n')
        while '\n \n' in product_description:
            product_description = product_description.replace('\n \n', '\n')
        # print('\n product_description_V4 \n', text_to_html6(handle_ordered_lists(product_description)))

        # company_description = remove_redundant_tags(company_description)
        while '  ' in company_description:
            company_description = company_description.replace('  ', ' ')
        while ' \n' in company_description:
            company_description = company_description.replace(' \n', '\n')
        while '\n ' in company_description:
            company_description = company_description.replace('\n ', '\n')
            
        while '<b>\n' in company_description:
            company_description = company_description.replace('<b>\n', '\n<b>')
        while '<b> </b>' in company_description:
            company_description = company_description.replace('<b> </b>', '')
        while '</b><b>' in company_description:
            company_description = company_description.replace('</b><b>', '')
        while '</b> <b>' in company_description:
            company_description = company_description.replace('</b> <b>', '')
        while '</i><i>' in company_description:
            company_description = company_description.replace('</i><i>', '')
        while '</i> <i>' in company_description:
            company_description = company_description.replace('</i> <i>', '')            

        while '\n</b>' in company_description:
            company_description = company_description.replace('\n</b>', '</b>\n')
        while '\n</i>' in company_description:
            company_description = company_description.replace('\n</i>', '</i>\n')
        while '\n</i></b>' in company_description:
            company_description = company_description.replace('\n</i></b>', '</i></b>\n')
        while '\n\n' in company_description:
            company_description = company_description.replace('\n\n', '\n')
        while '\n \n' in company_description:
            company_description = company_description.replace('\n \n', '\n')
        # print('\n company_description_V4 \n', text_to_html6(handle_ordered_lists(company_description)))
        print(i+1,'prod_descr1', product_description)


        # ------------------------------------------------------------------------------------------------------------------------------------------------------------------

        # global resultDict
        if 'companies' not in resultDict:
            resultDict['companies'] = {}
        # Add the company and product to the dictionary
        if company_name in resultDict['companies']:
            # If the dictionary exists, append 'element' to the list
            resultDict['companies'][company_name]['products'].append({
                'page':i+1, 
                'product_name': solution_name, 
                'product_description': text_to_html6(handle_ordered_lists(product_description)), 
                'target_audience':'',
                'scope': scope, 
                'destType':destType,
                'solType':solType})

            resultDict['companies'][company_name]['company_description'] = text_to_html6(handle_ordered_lists(company_description))
            resultDict['companies'][company_name]['company_url'] = company_url
            resultDict['companies'][company_name]['phone'] = ''
            resultDict['companies'][company_name]['email'] = ''
            # resultDict['companies'][company_name]['address'] = ''
            # resultDict['companies'][company_name]['geoRef'] = ''


            resultDict['companies'][company_name]['company_logo'] = company_logo
            resultDict['companies'][company_name]['company_QR'] = ''

          
        else:
            # If the dictionary doesn't exist, create a new one
            resultDict['companies'][company_name] = {
                'company_description': text_to_html6(handle_ordered_lists(company_description)),
                'company_url': company_url,
                'phone': '',
                'email': '',
                'address':'',
                'geoRef':'',                
                'company_logo': company_logo,
                'company_QR': '',
                
                'products': [
                    {
                        'page': i+1,
                        'product_name': solution_name,
                        'product_description': text_to_html6(handle_ordered_lists(product_description)),
                        'target_audience':'',
                        'scope': scope,
                        'destType': destType,
                        'solType': solType
                    }
                ]}
        print('page', i+1, 'done')
        # print('company_name', company_name, resultDict['companies'][company_name])
        # print('solution_name', solution_name)
        # print('company_url', company_url)
        # print('company_logo', company_logo)
        # print('company_description', company_description)
        # print('product_description', product_description)
        # print('scope', scope)
        # print('destType', destType)
        # print('solType', solType)

        
        # clear the memory for the next iteration
        del company_name, company_url, solution_name, scope, destType, solType, product_description, company_description, company_logo

    return resultDict

# result = get_data_html(pgList=range(9,363), pdfFile=pdfFile)
# result = get_data_html(pgList=range(46,363), pdfFile=pdfFile)
# result = get_data_html(pgList=range(84,363), pdfFile=pdfFile)
# result = get_data_html(pgList=range(119,363), pdfFile=pdfFile)
result = get_data_html(pgList=range(346,363), pdfFile=pdfFile)

# Save the results to a JSON file
with open('1stSpanish/3rdED/Catalogue Spanish_3.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [ ]:
# open 1stSpanish/3rdED/Catalogue Spanish_3.json
with open('1stSpanish/3rdED/Catalogue Spanish_3.json', 'r', encoding='utf-8') as f:
    result = json.load(f)

for key, val in result['companies'].items():
    print(key)
    for nestedKey, nestedVal in val.items():
        if nestedKey == 'products':
            for product in nestedVal:
                for product_key, product_val in product.items():
                    if product_val != '':
                        print(f'{product_key}: \n{product_val}')
        else:
            if nestedVal != '':
                print(f'{nestedKey}: \n{nestedVal}')

In [ ]:
def merge_sequential_tuples(tuples):
    merged = []
    current_start, current_end = tuples[0]

    for start, end in tuples[1:]:
        if start - 1 <= current_end:  # If the start of the next tuple is <= end of the current tuple + 1
            current_end = max(current_end, end)  # Extend the current tuple
        else:  # If the start of the next tuple is > end of the current tuple + 1
            merged.append((current_start, current_end))  # Add the current tuple to the merged list
            current_start, current_end = start, end  # Start a new tuple

    merged.append((current_start, current_end))  # Add the last tuple

    return merged

tuples = [(3, 7), (8, 8), (9, 10), (11, 11), (12, 13), (24,36),(36,38)]
print(merge_sequential_tuples(tuples))  # Output: [(3, 13), (24, 38)]



In [ ]:
from PIL import Image
import fitz
from IPython import display

doc = fitz.open(pdfFile)

# get page width and height
page_width = doc[0].bound().x1
page_height = doc[0].bound().y1

for i in range(9,363):
    page = doc[i]

    for block in page.get_text('dict')['blocks']:
        # draw a rectangle for each block['bbox'] in the page
        page.draw_rect(block['bbox'], color=(0, 0, 0), width=2)
        if fitz.Rect(600, 250, 900, 300).contains(fitz.Rect(*block['bbox'])):
            print('block', block)

        
    page.draw_rect([690, 0, page_width, page_height], color=(1, 0, 0), width=2)
    
    # Render the page to an image
    pix = page.get_pixmap()

    # Get the image data from the pixmap
    data = pix.samples

    # Create a PIL image from the data
    img = Image.frombytes("RGB", [pix.width, pix.height], data)

    # Display the image
    display.display(img)